In [1]:
import cobra
import copy

# Quicker way to read in models
import pickle
def read_model(fileName, obj='none'):
    
    fileType = fileName.split('.')[-1]
    
    if fileType == 'sbml' or fileType == 'xml':
        model = cobra.io.read_sbml_model(fileName)
    elif fileType == 'json':
        model = cobra.io.load_json_model(fileName)
    elif fileType == 'yaml':
        model = cobra.io.load_yaml_model(fileName)
    elif fileType == 'mat':
        model = cobra.io.load_matlab_model(fileName)
    elif fileType == 'pkl':
        model = pickle.load(open(fileName, 'rb'))
    else:
        raise TypeError('Unrecognized file extension')
    
    if obj != 'none': model.objective = obj
    for rxn in model.boundary: rxn.bounds = (-1000., 1000.)
        
    return model

In [2]:
# curated Clostridium difficile 630 model
iCdG719 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG719.json')

In [3]:
iCdG719

Name,iCdG719
Memory address,0x07fa785b35fd0
Number of metabolites,1216
Number of reactions,1277
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


In [4]:
len(iCdG719.genes)

719

In [5]:
iCdG719.slim_optimize()

190.16070481163646

In [223]:
# Create R20291 draft GENRE

import mackinac
mackinac.modelseed.ms_client.url = 'http://p3.theseed.org/services/ProbModelSEED/'
mackinac.workspace.ws_client.url = 'http://p3.theseed.org/services/Workspace'
mackinac.genome.patric_url = 'https://www.patricbrc.org/api/'

# PATRIC user information
mackinac.get_token('mljenior')
# password: matrix54

patric password: ········


'mljenior@patricbrc.org'

In [224]:
genome_id = '645463.3'
template_id = '/chenry/public/modelsupport/templates/GramPosModelTemplate'
media_id = '/chenry/public/modelsupport/media/Complete'
file_id = '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/PATRIC/CdR20291.draft.json'
strain_id = 'Clostridioides difficile R20291'

mackinac.reconstruct_modelseed_model(genome_id, template_reference=template_id)
mackinac.gapfill_modelseed_model(genome_id, media_reference=media_id)
mackinac.optimize_modelseed_model(genome_id)
patric_draft = mackinac.create_cobra_model_from_modelseed_model(genome_id)
patric_draft.id = strain_id

cobra.io.save_json_model(patric_draft, file_id)

In [6]:
# Get model gene IDs
iCdG719_genes = set([x.id for x in iCdG719.genes])
patric_draft = cobra.io.load_json_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/PATRIC/CdR20291.draft.json')
draft_R20291_genes = set([x.id for x in patric_draft.genes])

In [7]:
# Remove free biomass components
patric_draft.reactions.rxn13782_c.remove_from_model(remove_orphans=True)
patric_draft.reactions.rxn13784_c.remove_from_model(remove_orphans=True)

In [8]:
# Read in protein alignments

# 630 vs R20291
Cd630_geneDict = {}
R20291_missing = []
with open('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/proteome_comparisons/Cd630_R20291.tsv', 'r') as genes:
    header = genes.readline()
    for line in genes:
        Cd630_gene = line.split()[0]
        R20291_gene = line.split()[1]
        
        if not Cd630_gene in iCdG719_genes:
            continue
        elif R20291_gene == 'none':
            R20291_missing.append(Cd630_gene)
            continue
        else:
            Cd630_geneDict[Cd630_gene] = R20291_gene   

# R20291 vs 630
R20291_geneDict = {}
Cd630_missing = []
with open('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/proteome_comparisons/R20291_Cd630.tsv', 'r') as genes:
    header = genes.readline()
    for line in genes:
        R20291_gene = line.split()[0]
        Cd630_gene = line.split()[1]
        
        if Cd630_gene == 'none':
            Cd630_missing.append(R20291_gene)
            continue
        else:
            R20291_geneDict[R20291_gene] = Cd630_gene  
            
# Peptoclostridium 630 vs R20291
Pepto630_geneDict = {}
with open('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/proteome_comparisons/Pepto630_R20291.tsv', 'r') as genes:
    header = genes.readline()
    for line in genes:
        Pepto630_gene = line.split()[0]
        R20291_gene = line.split()[1]
        
        if not Pepto630_gene in iCdG719_genes:
            continue
        elif R20291_gene == 'none':
            R20291_missing.append(Pepto630_gene)
            continue
        else:
            Pepto630_geneDict[Pepto630_gene] = R20291_gene   

# R20291 vs Peptoclostridium 630
R20291_geneDict = {}
Pepto630_missing = []
with open('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/proteome_comparisons/R20291_Pepto630.tsv', 'r') as genes:
    header = genes.readline()
    for line in genes:
        R20291_gene = line.split()[0]
        Pepto630_gene = line.split()[1]
        
        if Cd630_gene == 'none':
            Pepto630_missing.append(R20291_gene)
            continue
        else:
            R20291_geneDict[R20291_gene] = Pepto630_gene  

# Create complete translation dictionary
geneDict = {}
geneDict.update(Cd630_geneDict)
geneDict.update(Pepto630_geneDict)

# Get gene objects to be removed
remove_genes = [iCdG719.genes.get_by_id(x) for x in set(R20291_missing)]

In [9]:
# Create duplicate GENRE
draft_GENRE = copy.deepcopy(iCdG719)

In [10]:
# Delete genes that are not present in R20291
remove_genes = [draft_GENRE.genes.get_by_id(x) for x in set(R20291_missing)]
cobra.manipulation.delete.remove_genes(draft_GENRE, remove_genes)

In [11]:
# Rename homologous genes
cobra.manipulation.modify.rename_genes(draft_GENRE, geneDict)

In [12]:
# Add genes that only appear in R20291
included_genes = list(set(draft_R20291_genes).intersection(set(Cd630_missing)))
current_rxn_ids = set([x.id for x in draft_GENRE.reactions])
rxns_added = 0
add_manually = []
for gene in patric_draft.genes:
    if not gene.id in included_genes: 
        add_manually.append(gene.id)
        continue
    
    for rxn in gene.reactions:
        if rxn.id in current_rxn_ids:
            # update existing gene_reaction_rule
            current_gpr = draft_GENRE.reactions.get_by_id(rxn.id).gene_reaction_rule
            if current_gpr != '':
                current_gpr += ' or ' + gene.id
            else:
                current_gpr = gene.id
            draft_GENRE.reactions.get_by_id(rxn.id).gene_reaction_rule = current_gpr
            # update gene name
            draft_GENRE.genes.get_by_id(gene.id).name = patric_draft.genes.get_by_id(gene.id).name 
        else:
            draft_GENRE.add_reactions([rxn])
            rxns_added += 1
            current_gpr = draft_GENRE.reactions.get_by_id(rxn.id).gene_reaction_rule
            draft_GENRE.reactions.get_by_id(rxn.id).gene_reaction_rule = current_gpr
            
print('Reactions added: ' + str(rxns_added))

Reactions added: 4


In [13]:
# Identify genes that must be added from the draft reconstruction
draft_genes = set([gene.id for gene in draft_GENRE.genes])
add_manually = set(add_manually).difference(draft_genes)
current_rxn_ids = set([x.id for x in draft_GENRE.reactions])

for gene in add_manually:
    for rxn in patric_draft.genes.get_by_id(gene).reactions:
        if rxn.id in current_rxn_ids:
            # update existing gene_reaction_rule
            current_gpr = draft_GENRE.reactions.get_by_id(rxn.id).gene_reaction_rule
            if current_gpr != '':
                current_gpr += ' or ' + gene
            else:
                current_gpr = gene
            draft_GENRE.reactions.get_by_id(rxn.id).gene_reaction_rule = current_gpr
            # update gene name
            draft_GENRE.genes.get_by_id(gene).name = patric_draft.genes.get_by_id(gene).name 
        else:
            draft_GENRE.add_reactions([rxn])
            draft_GENRE.genes.get_by_id(gene).name = patric_draft.genes.get_by_id(gene).name 
            rxns_added += 1

print('Reactions added: ' + str(rxns_added))

Ignoring reaction 'rxn13783_c' since it already exists.
Ignoring reaction 'rxn13783_c' since it already exists.
Ignoring reaction 'rxn13783_c' since it already exists.
Ignoring reaction 'rxn13783_c' since it already exists.
Ignoring reaction 'rxn10344_c' since it already exists.
Ignoring reaction 'rxn05516_c' since it already exists.
Ignoring reaction 'rxn10343_c' since it already exists.
Ignoring reaction 'rxn10481_c' since it already exists.
Ignoring reaction 'rxn05528_c' since it already exists.
Ignoring reaction 'rxn13783_c' since it already exists.
Ignoring reaction 'rxn13783_c' since it already exists.
Ignoring reaction 'rxn10344_c' since it already exists.
Ignoring reaction 'rxn05516_c' since it already exists.
Ignoring reaction 'rxn10343_c' since it already exists.
Ignoring reaction 'rxn10481_c' since it already exists.
Ignoring reaction 'rxn05528_c' since it already exists.
Ignoring reaction 'rxn13783_c' since it already exists.
Ignoring reaction 'rxn13783_c' since it already 

Reactions added: 46


In [14]:
# Fix a few remaining incorrect GPRs 
draft_GENRE.reactions.rxn02003_c.gene_reaction_rule = '645463.3.peg.237'
draft_GENRE.reactions.rxn14293_c.gene_reaction_rule += ' or 645463.3.peg.260'
draft_GENRE.reactions.rxn06517_c.gene_reaction_rule += ' or 645463.3.peg.1978'
draft_GENRE.reactions.rxn01404_c.gene_reaction_rule += ' or 645463.3.peg.658'
draft_GENRE.reactions.rxn01368_c.gene_reaction_rule += ' or 645463.3.peg.769'
draft_GENRE.reactions.rxn00119_c.gene_reaction_rule += ' or 645463.3.peg.2191'

In [15]:
# Replace removed Biomass component
cpd02967_c = draft_GENRE.metabolites.get_by_id('cpd02967_c') # N-Acetyl-beta-D-mannosaminyl-1,4-N-acetyl-D-glucosaminyldiphosphoundecaprenol
cpd00402_c = draft_GENRE.metabolites.get_by_id('cpd00402_c') # CDPglycerol
cpd00046_c = draft_GENRE.metabolites.get_by_id('cpd00046_c') # CMP

cpd12894_c = cobra.Metabolite(
    'cpd12894_c',
    formula='',
    name='Teichoic acid',
    compartment='cytosol')

# R20291 homolog unknown - Stabler et al. (2009). Genome Biol.
#gene = cobra.Gene('272563.8.peg.262')
#gene.name = 'Putative CDP-glycerol:Poly(Glycerophosphate) glycerophosphotransferase'

teichoicacid_rxn = cobra.Reaction('teichoicacid_rxn')
#teichoicacid_rxn.gene_reaction_rule = '272563.8.peg.262'
teichoicacid_rxn.name = 'Teichoic acid biosynthesis'
teichoicacid_rxn.lower_bound = 0.
teichoicacid_rxn.upper_bound = 1000.
teichoicacid_rxn.add_metabolites({
    cpd02967_c: -1.0,
    cpd00402_c: -1.0,
    cpd00046_c: 1.0,
    cpd12894_c: 1.0
})
draft_GENRE.add_reactions([teichoicacid_rxn])

In [16]:
# Replace Stickland pathway reaction
cpd00005_c = draft_GENRE.metabolites.get_by_id('cpd00005_c') # NADPH
cpd00067_c = draft_GENRE.metabolites.get_by_id('cpd00067_c') # H+
cpd00498_c = draft_GENRE.metabolites.get_by_id('cpd00498_c') # 2-Aceto-2-hydroxybutanoate
cpd00006_c = draft_GENRE.metabolites.get_by_id('cpd00006_c') # NADP
cpd02535_c = draft_GENRE.metabolites.get_by_id('cpd02535_c') # 2,3-Dihydroxy-3-methylvalerate

gene = cobra.Gene('645463.3.peg.1493')
gene.name = 'Ketol-acid reductoisomerase (NADP(+)) (EC 1.1.1.86)'

rxn08764_c = cobra.Reaction('rxn08764_c')
rxn08764_c.gene_reaction_rule = '645463.3.peg.1493'
rxn08764_c.name = 'Teichoic acid biosynthesis'
rxn08764_c.lower_bound = -1000.
rxn08764_c.upper_bound = 1000.
rxn08764_c.add_metabolites({
    cpd00005_c: -1.0,
    cpd00067_c: -1.0,
    cpd00498_c: -1.0,
    cpd00006_c: 1.0,
    cpd02535_c: 1.0
})
draft_GENRE.add_reactions([rxn08764_c])

In [17]:
# Remove DNA demand
try:
    draft_GENRE.reactions.get_by_id('rxn13783_c').remove_from_model(remove_orphans=True)
except:
    pass

In [18]:
# Prune orphaned nodes
cpd_total = 0
rxn_total = 0
removed = 1
while removed == 1:
    removed = 0

    # Metabolites
    for cpd in draft_GENRE.metabolites:
        if len(cpd.reactions) == 0:
            cpd.remove_from_model()
            removed = 1
            cpd_total += 1

    # Reactions
    for rxn in draft_GENRE.reactions:
        if len(rxn.metabolites) == 0: 
            rxn.remove_from_model()
            removed = 1
            rxn_total += 1
    
print('Metabolites pruned: ' + str(cpd_total))
print('Reactions pruned: ' + str(rxn_total))

Metabolites pruned: 2
Reactions pruned: 0


In [19]:
# Fix compartments
for cpd in draft_GENRE.metabolites:
    if cpd.compartment == 'c':
        cpd.compartment = 'cytosol'
    elif cpd.compartment == 'e':
        cpd.compartment = 'extracellular'
    elif cpd.compartment == 'p':
        cpd.compartment = 'periplasm'

In [20]:
# Open exchange bounds
for rxn in draft_GENRE.boundary:
    rxn.bounds = (-1000.0,1000.0)

In [21]:
# Test that new model grows
draft_GENRE.slim_optimize()

190.16070481163598

In [22]:
# Test growth in minimal media conditions

# Karlsson et al. (1999). Microbiology.
mdm = ['cpd00001_e', # water
       'cpd00065_e', # L-Tryptophan
       'cpd00060_e', # L-Methionine
       'cpd00322_e', # L-Isoleucine
       'cpd00129_e', # L-Proline
       'cpd00156_e', # L-Valine
       'cpd00107_e', # L-Leucine
       'cpd00084_e', # L-Cysteine 
       'cpd00149_e', # Cobalt
       'cpd00099_e', # Chloride
       'cpd10515_e', # Iron
       'cpd00030_e', # Manganese
       'cpd00254_e', # Magnesium
       'cpd00063_e', # Calcium
       'cpd00205_e', # Potassium
       'cpd00009_e', # Phosphate
       'cpd00971_e', # Sodium
       'cpd00242_e', # Carbonate
       'cpd00104_e', # Biotin
       'cpd00644_e', # Pantothenate
       'cpd00263_e', # Pyridoxine
       'cpd00027_e'] # D-Glucose (Carbohydrate C-source)

# Theriot et al. (2013). Nature Communications.
ncmm = ['cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine 
        'cpd00065_e', # L-Tryptophan
        'cpd00060_e', # L-Methionine
        'cpd00119_e', # L-Histidine
        'cpd00033_e', # Glycine
        'cpd00051_e', # L-Arginine
        'cpd00161_e'] # L-Threonine

ncmm_exchanges = ['EX_' + x for x in ncmm]
for rxn in draft_GENRE.boundary:
    if not rxn.id in ncmm_exchanges:
        rxn.bounds = (0.0,1000.0)
    else:
        rxn.bounds = (-1000.0,1000.0)
print('NCMM: ' + str(draft_GENRE.slim_optimize()))

mdm_exchanges = ['EX_' + x for x in mdm]
for rxn in draft_GENRE.boundary:
    if not rxn.id in mdm_exchanges:
        rxn.bounds = (0.0,1000.0)
    else:
        rxn.bounds = (-1000.0,1000.0)
print('MDM: ' + str(draft_GENRE.slim_optimize()))

# Leave in MDM

NCMM: 56.14277852157972
MDM: 80.96607010298253


In [24]:
for x in draft_GENRE.genes:
    if len(x.reactions) == 0:
        print(x.id)

In [23]:
# Remove genes with no reactions
remove_genes = [draft_GENRE.genes.get_by_id(x) for x in ['645463.3.peg.1041','645463.3.peg.1043','645463.3.peg.1040','645463.3.peg.1042','645463.3.peg.1045','645463.3.peg.1044']]
cobra.manipulation.delete.remove_genes(draft_GENRE, remove_genes)

KeyError: '645463.3.peg.1041'

In [25]:
# Name new model
draft_GENRE.name = 'Clostridium difficile R20291'
draft_GENRE.id = 'iCdR' + str(len(draft_GENRE.genes))

In [26]:
draft_GENRE

Name,iCdR700
Memory address,0x07fa79f4d7a50
Number of metabolites,1238
Number of reactions,1292
Number of groups,0
Objective expression,0.0 + 1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


In [27]:
# Add annotation information

# SBO designations
# Metabolites
for cpd in draft_GENRE.metabolites:
    cpd.annotation['sbo'] = 'SBO:0000247'
# Reactions
for rxn in draft_GENRE.reactions:
    substrates = list(rxn.metabolites)
    compartments = set([x.compartment for x in substrates])
    if 'EX_' in rxn.id:
        rxn.annotation['sbo'] = 'SBO:0000627' # exchange
    elif len(compartments) > 1:
        rxn.annotation['sbo'] = 'SBO:0000185' # transport
    else:
        rxn.annotation['sbo'] = 'SBO:0000176' # metabolic
# Biomass
draft_GENRE.reactions.dna_rxn.annotation['sbo'] = 'SBO:0000629'  
draft_GENRE.reactions.rna_rxn.annotation['sbo'] = 'SBO:0000629'  
draft_GENRE.reactions.protein_rxn.annotation['sbo'] = 'SBO:0000629'  
draft_GENRE.reactions.teichoicacid_rxn.annotation['sbo'] = 'SBO:0000629'  
draft_GENRE.reactions.peptidoglycan_rxn.annotation['sbo'] = 'SBO:0000629'  
draft_GENRE.reactions.cellwall_rxn.annotation['sbo'] = 'SBO:0000629'  
draft_GENRE.reactions.lipid_rxn.annotation['sbo'] = 'SBO:0000629'  
draft_GENRE.reactions.cofactor_rxn.annotation['sbo'] = 'SBO:0000629'  
draft_GENRE.reactions.biomass.annotation['sbo'] = 'SBO:0000629'  
draft_GENRE.reactions.SK_cpd11416_c.annotation['sbo'] = 'SBO:0000632'  
# Genes
for gene in draft_GENRE.genes:
    gene.annotation['sbo'] = 'SBO:0000243'

# RefSeq annotations
patric_refseq = {}
with open('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/r20291_refseq_genes.tsv') as ref:
    for line in ref:
        patric_refseq[line.split()[0]] = line.split()[1]
for gene in draft_GENRE.genes:
    if '.peg.' in gene.id:
        gene.annotation['refseq'] = 'NC_013316.1'
        try:
            gene.annotation['refseq.locustag'] = patric_refseq[gene.id]
        except:
            continue

# Metabolites
for cpd in draft_GENRE.metabolites:
    if 'cpd' in cpd.id:
        new_id = str(cpd.id).split('_')
        if len(new_id) > 1:
            del new_id[-1]
        new_id = ''.join(new_id)
        cpd.annotation['seed.compound'] = new_id
draft_GENRE.metabolites.C21400_c.annotation['kegg.compound'] = 'C21400'        
draft_GENRE.metabolites.C21399_c.annotation['kegg.compound'] = 'C21399'        
draft_GENRE.metabolites.C21090_c.annotation['kegg.compound'] = 'C21090'        
draft_GENRE.metabolites.C21399_e.annotation['kegg.compound'] = 'C21399'        

# Reactions
for rxn in draft_GENRE.reactions:
    if 'rxn' in rxn.id or 'EX_' in rxn.id:
        new_id = str(rxn.id).split('_')
        if len(new_id) > 1:
            del new_id[-1]
        new_id = ''.join(new_id)
        rxn.annotation['seed.reaction'] = new_id
    elif 'R' in rxn.id:
        new_id = str(rxn.id).split('_')
        if len(new_id) > 1:
            del new_id[-1]
        new_id = ''.join(new_id)
        rxn.annotation['kegg.reaction'] = new_id
draft_GENRE.reactions.SK_cpd11416_c.annotation['seed.reaction'] = 'SK_cpd11416'
draft_GENRE.reactions.ENOG4108HXH_c.annotation['eggnog.reaction'] = 'ENOG4108HXH'
draft_GENRE.reactions.COG3601_c.annotation['cog.reaction'] = 'COG3601'
draft_GENRE.reactions.K20025_c.annotation['kegg.reaction'] = 'K20025'

# Genes
for gene in draft_GENRE.genes:
    if '.peg.' in gene.id:
        gene.annotation['patric'] = 'fig|' + gene.id

In [28]:
# Correct residual gene names
draft_GENRE.genes.get_by_id('645463.3.peg.1176').name = 'Na(+)-dependent branched-chain amino acid transporter'
draft_GENRE.genes.get_by_id('645463.3.peg.735').name = 'ABC transporter, substrate-binding protein'
draft_GENRE.genes.get_by_id('645463.3.peg.2463').name = 'PTS system, glucitol/sorbitol-specific IIB component'
draft_GENRE.genes.get_by_id('645463.3.peg.929').name = 'N-acetylgalactosamine-6-phosphate deacetylase'
draft_GENRE.genes.get_by_id('645463.3.peg.2324').name = 'ribulose-5-phosphate 4-epimerase'
draft_GENRE.genes.get_by_id('645463.3.peg.46').name = 'Ribulose-5-phosphate 4-epimerase'
draft_GENRE.genes.get_by_id('645463.3.peg.3015').name = 'PTS system, IIB component'
draft_GENRE.genes.get_by_id('645463.3.peg.3014').name = 'PTS system, IIC component'
draft_GENRE.genes.get_by_id('645463.3.peg.3016').name = 'PTS system, IIA component'
draft_GENRE.genes.get_by_id('645463.3.peg.865').name = 'ABC transporter, substrate-binding protein'
draft_GENRE.genes.get_by_id('645463.3.peg.862').name = 'ABC transporter, substrate-binding protein'
draft_GENRE.genes.get_by_id('645463.3.peg.664').name = 'sodium-solute symporter, putative'
draft_GENRE.genes.get_by_id('645463.3.peg.1783').name = 'ABC transporter, ATP-binding protein'
draft_GENRE.genes.get_by_id('645463.3.peg.1421').name = 'DNA-3-methyladenine glycosylase II'
draft_GENRE.genes.get_by_id('645463.3.peg.2254').name = '2-oxoglutarate/2-oxoacid ferredoxin oxidoreductase, delta subunit'
draft_GENRE.genes.get_by_id('645463.3.peg.2253').name = '2-oxoglutarate/2-oxoacid ferredoxin oxidoreductase, alpha subunit'
draft_GENRE.genes.get_by_id('645463.3.peg.168').name = '4-hydroxyphenylacetate decarboxylase, small subunit'
draft_GENRE.genes.get_by_id('645463.3.peg.167').name = '4-hydroxyphenylacetate decarboxylase, large subunit'
draft_GENRE.genes.get_by_id('645463.3.peg.2417').name = 'Indolepyruvate oxidoreductase subunit'
draft_GENRE.genes.get_by_id('645463.3.peg.2389').name = 'Glycine reductase component B beta subunit'
draft_GENRE.genes.get_by_id('645463.3.peg.392').name = '2-hydroxyglutaryl-CoA dehydratase, A-component'
draft_GENRE.genes.get_by_id('645463.3.peg.393').name = '2-hydroxyglutaryl-CoA dehydratase, B-component'
draft_GENRE.genes.get_by_id('645463.3.peg.390').name = 'Hydroxyproline dehydratase putative'
draft_GENRE.genes.get_by_id('645463.3.peg.391').name = 'CoA-substrate-specific enzyme activase; 2-hydroxyglutaryl-CoA dehydratase activator, A-component'
draft_GENRE.genes.get_by_id('645463.3.peg.1466').name = 'NADH-dependent reduced ferredoxin:NADP+ oxidoreductase subunit A'
draft_GENRE.genes.get_by_id('645463.3.peg.169').name = '4-hydroxyphenylacetate decarboxylase activating enzyme'
draft_GENRE.genes.get_by_id('645463.3.peg.2250').name = 'Acetyl-CoA synthetase (ADP-forming) alpha and beta chains, putative'
draft_GENRE.genes.get_by_id('645463.3.peg.2379').name = 'Succinate-semialdehyde dehydrogenase, CoA-dependent'
draft_GENRE.genes.get_by_id('645463.3.peg.976').name = 'Electron bifurcating butyryl-CoA dehydrogenase (NAD+, ferredoxin)'
draft_GENRE.genes.get_by_id('645463.3.peg.2703').name = 'UDP-N-acetylmuramoyl-tripeptide--D-alanyl-D-alanine ligase'
draft_GENRE.genes.get_by_id('645463.3.peg.1493').name = 'Ketol-acid reductoisomerase (NADP(+)) (EC 1.1.1.86) | ilvC'

In [29]:
# Add additional annotation information

# BiGG
BiGG_rxn_ids = {}
with open('/home/mjenior/Desktop/repos/rxns_BiGG_ids.txt', 'r') as bigg_rxns:
    for line in bigg_rxns:
        line = line.split('=')
        modelseed = line[0]
        bigg = line[1].strip().split(',')
        if len(bigg) == 1: bigg = bigg[0]
        BiGG_rxn_ids[modelseed] = bigg
BiGG_cpd_ids = {}
with open('/home/mjenior/Desktop/repos/cpds_BiGG_ids.txt', 'r') as bigg_rxns:
    for line in bigg_rxns:
        line = line.split('=')
        modelseed = line[0]
        bigg = line[1].strip().split(',')
        if len(bigg) == 1: bigg = bigg[0]
        BiGG_cpd_ids[modelseed] = bigg

# KEGG
KEGG_rxn_ids = {}
with open('/home/mjenior/Desktop/repos/rxns_KEGG_ids.txt', 'r') as kegg_rxns:
    for line in kegg_rxns:
        line = line.split('=')
        modelseed = line[0]
        kegg = line[1].strip().split(',')
        if len(kegg) == 1: kegg = kegg[0]
        KEGG_rxn_ids[modelseed] = kegg
KEGG_cpd_ids = {}
with open('/home/mjenior/Desktop/repos/cpds_KEGG_ids.txt', 'r') as kegg_rxns:
    for line in kegg_rxns:
        line = line.split('=')
        modelseed = line[0]
        kegg = line[1].strip().split(',')
        if len(kegg) == 1: kegg = kegg[0]
        KEGG_cpd_ids[modelseed] = kegg

# Add annotations
bigg_rxn_success = 0
bigg_cpd_success = 0
kegg_rxn_success = 0
kegg_cpd_success = 0

for rxn in draft_GENRE.reactions:
    new_id = rxn.id.split('_')[0]
    try:
        bigg_id = BiGG_rxn_ids[new_id]
        draft_GENRE.reactions.get_by_id(rxn.id).annotation['bigg.metabolite'] = bigg_id
        bigg_rxn_success += 1
    except:
        pass
    try:
        kegg_id = KEGG_rxn_ids[new_id]
        draft_GENRE.reactions.get_by_id(rxn.id).annotation['kegg.compound'] = kegg_id
        kegg_rxn_success += 1
    except:
        pass

for cpd in draft_GENRE.metabolites:
    new_id = cpd.id.split('_')[0]
    try:
        bigg_id = BiGG_cpd_ids[new_id]
        draft_GENRE.metabolites.get_by_id(cpd.id).annotation['bigg.metabolite'] = bigg_id
        bigg_cpd_success += 1
    except:
        pass
    try:
        kegg_id = KEGG_cpd_ids[new_id]
        draft_GENRE.metabolites.get_by_id(cpd.id).annotation['kegg.compound'] = kegg_id
        kegg_cpd_success += 1
    except:
        pass

In [30]:
# Add additional gene annotations

uniprot_gene_ids = {}
ec_gene_ids = {}
refseq_gene_ids = {}
kegg_gene_ids = {}
patric_genes = set()

with open('/home/mjenior/Desktop/repos/uniprot_CdR20291.tab', 'r') as gene_annotation:
    header = gene_annotation.readline()
    for line in gene_annotation:
        line = line.split()
        uniprot_gene_ids[line[4]] = line[0]
        ec_gene_ids[line[4]] = line[1]
        refseq_gene_ids[line[4]] = line[2]
        kegg_gene_ids[line[4]] = line[3]
        patric_genes |= set([line[4]])

success = 0
for gene in draft_GENRE.genes:
    if not gene.id in patric_genes: continue
    
    success += 1
    gene.annotation['uniprot'] = uniprot_gene_ids[gene.id]
    gene.annotation['kegg.genes'] = kegg_gene_ids[gene.id]
    
    if refseq_gene_ids[gene.id] != 'none':
        gene.annotation['refseq'] = refseq_gene_ids[gene.id]
    
    if ec_gene_ids[gene.id] != 'none':
        gene.annotation['ec_number'] = ec_gene_ids[gene.id]
    


In [31]:
# Unblock Lactose catabolism

cpd00208_c = draft_GENRE.metabolites.cpd00208_c # Lactose
cpd00001_c = draft_GENRE.metabolites.cpd00001_c # H2O
cpd00027_c = draft_GENRE.metabolites.cpd00027_c # Glucose

cpd00108_c = cobra.Metabolite(
    'cpd00108_c',
    formula='C6H12O6',
    name='Galactose',
    compartment='cytosol')
cpd00108_c.annotation['sbo'] = 'SBO:0000247'
cpd00108_c.annotation['seed.compound'] = 'cpd00108'
cpd00108_c.annotation['bigg.metabolite'] = 'gal'
cpd00108_c.annotation['kegg.compound'] = 'C00124'

rxn00816_c = cobra.Reaction('rxn00816_c')
rxn00816_c.name = 'Beta-galactosidase'
rxn00816_c.lower_bound = 0.
rxn00816_c.upper_bound = 1000.
rxn00816_c.add_metabolites({
    cpd00208_c: -1.0,
    cpd00001_c: -1.0,
    cpd00027_c: 1.0,
    cpd00108_c: 1.0
})
rxn00816_c.annotation['seed.reaction'] = 'rxn00816'
rxn00816_c.annotation['kegg.reaction'] = 'R01100'
rxn00816_c.annotation['bigg.reaction'] = 'LACZ'
draft_GENRE.add_reactions([rxn00816_c])

cpd00002_c = draft_GENRE.metabolites.cpd00002_c # ATP
cpd00008_c = draft_GENRE.metabolites.cpd00008_c # ADP
cpd00067_c = draft_GENRE.metabolites.cpd00067_c # H
cpd00348_c = draft_GENRE.metabolites.cpd00348_c # galactose-1-phosphate 

rxn00808_c = cobra.Reaction('rxn00808_c')
rxn00808_c.name = 'ATP:D-galactose 1-phosphotransferase'
rxn00808_c.lower_bound = 0.
rxn00808_c.upper_bound = 1000.
rxn00808_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00108_c: -1.0,
    cpd00008_c: 1.0,
    cpd00067_c: 1.0,
    cpd00348_c: 1.0
})
rxn00808_c.annotation['seed.reaction'] = 'rxn00808'
rxn00808_c.annotation['kegg.reaction'] = 'R01092'
rxn00808_c.annotation['bigg.reaction'] = 'GAL1'
draft_GENRE.add_reactions([rxn00808_c])


Ignoring reaction 'rxn00816_c' since it already exists.
Ignoring reaction 'rxn00808_c' since it already exists.


In [33]:
# Unblock Salicin catabolism

cpd01553_c = draft_GENRE.metabolites.cpd01553_c # Saligenin

cpd01553_e = cobra.Metabolite(
    'cpd01553_e',
    formula='C7H8O2',
    name='Saligenin',
    compartment='extracellular')
cpd01553_e.annotation['sbo'] = 'SBO:0000247'
cpd01553_e.annotation['seed.compound'] = 'cpd01553'
cpd01553_e.annotation['bigg.metabolite'] = '2hymeph'
cpd01553_e.annotation['kegg.compound'] = 'C02323'
draft_GENRE.add_boundary(cpd01553_e, type='exchange', reaction_id='EX_cpd01553_e', lb=-1000.0, ub=1000.0)
draft_GENRE.reactions.EX_cpd01553_e.name = 'Saligenin exchange'
draft_GENRE.reactions.EX_cpd01553_e.annotation['sbo'] = 'SBO:0000627'

cpd00067_c = draft_GENRE.metabolites.cpd00067_c # H
cpd00067_e = draft_GENRE.metabolites.cpd00067_e # H

rxn05472_c = cobra.Reaction('rxn05472_c')
rxn05472_c.name = '2-(hydroxymethyl)phenol transport in/out via proton symport'
rxn05472_c.lower_bound = -1000.
rxn05472_c.upper_bound = 1000.
rxn05472_c.add_metabolites({
    cpd00067_e: -1.0,
    cpd01553_e: -1.0,
    cpd00067_c: 1.0,
    cpd01553_c: 1.0
})
rxn05472_c.annotation['sbo'] = 'SBO:0000185'
rxn05472_c.annotation['seed.reaction'] = 'rxn05472'
rxn05472_c.annotation['bigg.reaction'] = '2HXMPt6'
draft_GENRE.add_reactions([rxn05472_c])


Ignoring reaction 'rxn05472_c' since it already exists.


In [35]:
# Correct Melezitose catabolism

cpd00001_c = draft_GENRE.metabolites.cpd00001_c # H2O
cpd05161_c = draft_GENRE.metabolites.cpd05161_c # Melezitose
cpd00027_c = draft_GENRE.metabolites.cpd00027_c # Glucose
cpd20885_c = draft_GENRE.metabolites.cpd20885_c # Turanose

R01103_c = cobra.Reaction('R01103_c')
R01103_c.name = 'Melezitose galactohydrolase'
R01103_c.lower_bound = 0.
R01103_c.upper_bound = 1000.
R01103_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd05161_c: -1.0,
    cpd00027_c: 1.0,
    cpd20885_c: 1.0
})
R01103_c.annotation['kegg.reaction'] = 'R01103'
R01103_c.annotation['sbo'] = 'SBO:0000176'
draft_GENRE.add_reactions([R01103_c])

draft_GENRE.id = 'iCdR' + str(len(draft_GENRE.genes))

Ignoring reaction 'R01103_c' since it already exists.


In [36]:
# Set media condition
for rxn in draft_GENRE.boundary:
    if not rxn.id in mdm_exchanges:
        rxn.bounds = (0.0,1000.0)
    else:
        rxn.bounds = (-1000.0,1000.0)

In [37]:
draft_GENRE.reactions.EX_cpd00104_e.name = 'Biotin exchange'
draft_GENRE.reactions.rxn05872_c.name = 'Secondary alcohol:NAD+ oxidoreductase'
draft_GENRE.reactions.EX_cpd00208_e.name = 'Lactose exchange'

In [38]:
# Label tRNA reactions as Biomass
tRNA = ["rxn06449_c","rxn06443_c","rxn06444_c","rxn06445_c","rxn06446_c","rxn06447_c","rxn06448_c","rxn06280_c","rxn06300_c","rxn06432_c","rxn06434_c","rxn06936_c","rxn06435_c","rxn06437_c","rxn06436_c","rxn06438_c","rxn06439_c","rxn06440_c","rxn06441_c","rxn06442_c"]
for rxn in tRNA: draft_GENRE.reactions.get_by_id(rxn).annotation['sbo'] = 'SBO:0000629'

In [39]:
draft_GENRE

Name,iCdR700
Memory address,0x07fa79f4d7a50
Number of metabolites,1238
Number of reactions,1292
Number of groups,0
Objective expression,0.0 + 1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


In [1]:
import cobra
from cobra.flux_analysis import parsimonious

# Function to calculate doubling time from objective value
def doublingTime(model):
    with model as m:
        if m.slim_optimize(error_value=0.) < 1e-6:
            print('GENRE has no objective flux')
        else:
            growth = (1. / float(m.slim_optimize())) * 3600.
            print(str(round(growth, 2)) + ' minutes doubling time')


# Identifies blocked reactions, 1% cutoff for fraction of optimum
def blockedReactions(model):
    
    with model as m:
        blocked = cobra.flux_analysis.variability.find_blocked_reactions(m)
        nogene_blocked = []
        for rxn in blocked:
            if m.reactions.get_by_id(rxn).gene_reaction_rule == '':
                nogene_blocked.append(rxn)

    #print(str(len(blocked)) + ' total reactions are blocked')
    fraction = (float(len(blocked)) / float(len(model.reactions))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% reactions are blocked')
    
    return blocked


# Identify potentially gapfilled reactions, checks against pFBA solution
def missingGPR(model):
    
    noGene = []
    exclude = []
    for rxn in model.reactions:
        if len(list(rxn.genes)) == 0:
            if rxn.annotation['sbo'] != 'SBO:0000629':
                if rxn in model.boundary:
                    exclude.append(rxn.id)
                    continue
                else:
                    noGene.append(rxn.id)
    
    solution = parsimonious.pfba(model)
    active_rxns = set([rxn.id for rxn in model.reactions if abs(solution.fluxes[rxn.id]) > 1e-5])
    active_rxns = active_rxns.difference(set(exclude))
    noGene_active = set(noGene).intersection(active_rxns)

    fraction = float(len(model.reactions)) - float(len(exclude))
    fraction = (float(len(noGene)) / fraction) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% reactions without GPRs')
    
    fraction = (float(len(noGene_active)) / float(len(active_rxns))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% of reactions used in pFBA solution have no GPR')
    
    return noGene_active


# Checks which cytosolic metabolites are generated for free (bacteria only)
def checkFreeMass(model, cytosol='cytosol'):

    free = []
    with model as m:
    
        # Close all exchanges
        for rxn in m.boundary: m.reactions.get_by_id(rxn.id).lower_bound = 0.
    
        # Create demand for each reaction and optimize individually
        reset_rxn = m.reactions[0].id
        for cpd in m.metabolites: 
            if cpd.compartment == cytosol:
                demand = cobra.Reaction('demand')
                demand.bounds = (0., 1000.)
                demand.add_metabolites({cpd: -1.0})
                m.add_reactions([demand])
                m.objective = demand
                obj_val = m.slim_optimize()
                if obj_val > 1e-8: free.append(cpd.id)
                m.objective = reset_rxn
                m.remove_reactions([demand])
    
    fraction = (float(len(free)) / float(len(model.metabolites))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% metabolites are generated for free')

    return(free)


# Check for mass and charge balance in reactions
def checkBalance(model):
    
    with model as m:

        elements = set()
        for cpd in m.metabolites:
            try:
                elements |= set(cpd.elements.keys())
            except:
                pass
        
        massImbal = []
        failed = 0
        if len(elements) == 0:
            print('No elemental data associated with metabolites!')
            failed = 1
        else:
            for rxn in m.reactions:
                if rxn.annotation['sbo'] == 'SBO:0000629': 
                    continue
                elif rxn in m.boundary:
                    continue

                try:
                    test = rxn.check_mass_balance()
                except ValueError:
                    continue

                if len(list(test)) > 0:
                    if len(set(test.keys()).intersection(elements)) > 0: massImbal.append(rxn.id)
                        
    if failed != 1:
        fraction = (float(len(massImbal)) / float(len(model.reactions))) * 100.
        fraction = round(fraction, 2)
        print(str(fraction) + '% reactions are mass imbalanced')
        
    return massImbal


def basicCheck(model):
    
    # Determination
    if len(model.reactions) < len(model.metabolites): 
        print('GENRE is overdetermined')
    elif len(model.reactions) > len(model.metabolites):
        print('GENRE is underdetermined')
    else:
        pass
    
    # Compartments
    print('GENRE has ' + str(len(model.compartments.keys())) + ' compartment(s)')
    
    # Genes
    if len(model.genes) == 0: 
        print('GENRE has no gene data')
    else:
        print('GENRE has ' + str(len(model.genes)) + ' genes')
          
    # Growth
    doublingTime(model)

# Quicker way to read in models
import pickle
def read_model(fileName, obj='none'):
    
    fileType = fileName.split('.')[-1]
    
    if fileType == 'sbml' or fileType == 'xml':
        model = cobra.io.read_sbml_model(fileName)
    elif fileType == 'json':
        model = cobra.io.load_json_model(fileName)
    elif fileType == 'yaml':
        model = cobra.io.load_yaml_model(fileName)
    elif fileType == 'mat':
        model = cobra.io.load_matlab_model(fileName)
    elif fileType == 'pkl':
        model = pickle.load(open(fileName, 'rb'))
    else:
        raise TypeError('Unrecognized file extension')
    
    if obj != 'none': model.objective = obj
    for rxn in model.boundary: rxn.bounds = (-1000., 1000.)
        
    return model


def save_model(model, fileName, media='none'):
    
    if media != 'none':
        #media_exchanges = ['EX_' + x for x in media]
        for rxn in model.boundary:
            if not rxn.id in media:
                rxn.bounds = (0., 1000.)
            else:
                rxn.bounds = (-1000., 1000.)
    else:
        for rxn in model.boundary:
            rxn.bounds = (-1000., 1000.)
    
    sbml_fileName = fileName + '.sbml'
    cobra.io.write_sbml_model(model, sbml_fileName)
    
    json_fileName = fileName + '.json'
    cobra.io.save_json_model(model, json_fileName)
    
mdm = ['cpd00001_e', # water
       'cpd00065_e', # L-Tryptophan
       'cpd00060_e', # L-Methionine
       'cpd00322_e', # L-Isoleucine
       'cpd00129_e', # L-Proline
       'cpd00156_e', # L-Valine
       'cpd00107_e', # L-Leucine
       'cpd00084_e', # L-Cysteine 
       'cpd00149_e', # Cobalt
       'cpd00099_e', # Chloride
       'cpd10515_e', # Iron
       'cpd00030_e', # Manganese
       'cpd00254_e', # Magnesium
       'cpd00063_e', # Calcium
       'cpd00205_e', # Potassium
       'cpd00009_e', # Phosphate
       'cpd00971_e', # Sodium
       'cpd00242_e', # Carbonate
       'cpd00104_e', # Biotin
       'cpd00644_e', # Pantothenate
       'cpd00263_e', # Pyridoxine
       'cpd00027_e'] # D-Glucose (Carbohydrate C-source)


In [36]:
basicCheck(draft_GENRE)
draft_noGPRblocked = blockedReactions(draft_GENRE)
draft_free = checkFreeMass(draft_GENRE)
draft_massImbal = checkBalance(draft_GENRE)
draft_nogene = missingGPR(draft_GENRE)

GENRE is underdetermined
GENRE has 3 compartment(s)
GENRE has 700 genes
51.39 minutes doubling time
48.94% reactions are blocked
0.0% metabolites are generated for free
0.0% reactions are mass imbalanced
12.66% reactions without GPRs
22.09% of reactions used in pFBA solution have no GPR


In [41]:
# Save to files
cobra.io.write_sbml_model(draft_GENRE, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.sbml')
cobra.io.save_json_model(draft_GENRE, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.json')

In [44]:

iCdR700 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.json')

# Neu5Ac --> N-Acetyl-Neuraminic acid
iCdR700.metabolites.cpd00232_c.name = 'N-Acetyl-Neuraminic acid'

# D-Tagatose
cpd00589_e = cobra.Metabolite(
    'cpd00589_e',
    formula='C6H12O6',
    name='D-Tagatose',
    compartment='extracellular')
cpd00589_e.annotation['sbo'] = 'SBO:0000247'
cpd00589_e.annotation['seed.compound'] = 'cpd00589'
cpd00589_e.annotation['bigg.metabolite'] = 'tag__D'
cpd00589_e.annotation['kegg.compound'] = 'C00795'
#iCdR700.add_boundary(cpd00589_e, type='exchange', reaction_id='EX_cpd00589_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd01553_e.name = 'Tagatose exchange'
iCdR700.reactions.EX_cpd01553_e.annotation['sbo'] = 'SBO:0000627'

cpd00805_c = iCdR700.metabolites.cpd00805_c # D-Tagatose 6-phosphate
cpd00002_c = iCdR700.metabolites.cpd00002_c # ATP
cpd00008_c = iCdR700.metabolites.cpd00008_c # ADP
cpd00067_c = iCdR700.metabolites.cpd00067_c # H

rxn02094_c = cobra.Reaction('rxn02094_c')
rxn02094_c.name = 'ATP:D-tagatose 6-phosphotransferase'
rxn02094_c.gene_reaction_rule = '( 645463.3.peg.3332 or 645463.3.peg.303 or 645463.3.peg.464 or 645463.3.peg.998 ) and ( 645463.3.peg.306 or 645463.3.peg.465 or 645463.3.peg.3333 or 645463.3.peg.1000 or 645463.3.peg.304 ) and ( 645463.3.peg.307 or 645463.3.peg.3331 or 645463.3.peg.466 or 645463.3.peg.1001 ) and ( 645463.3.peg.467 or 645463.3.peg.1002 or 645463.3.peg.308 or 645463.3.peg.3330 )'
rxn02094_c.lower_bound = 0.
rxn02094_c.upper_bound = 1000.
rxn02094_c.add_metabolites({
    cpd00589_e: -1.0,
    cpd00002_c: -1.0,
    cpd00008_c: 1.0,
    cpd00067_c: 1.0,
    cpd00805_c: 1.0
})
rxn02094_c.annotation['sbo'] = 'SBO:0000185' # transport
rxn02094_c.annotation['seed.reaction'] = 'rxn02094'
rxn02094_c.annotation['kegg.reaction'] = 'R02927'
rxn02094_c.annotation['bigg.reaction'] = 'TAGptspp'
iCdR700.add_reactions([rxn02094_c])


# N-Acetyl-D-Galactosamine
cpd27607_e = cobra.Metabolite(
    'cpd27607_e',
    formula='C8H15NO6',
    name='N-Acetyl-D-Galactosamine',
    compartment='extracellular')
cpd27607_e.annotation['sbo'] = 'SBO:0000247'
cpd27607_e.annotation['seed.compound'] = 'cpd27607'
cpd27607_e.annotation['bigg.metabolite'] = 'acgal'
cpd27607_e.annotation['kegg.compound'] = 'C01074'

#iCdR700.add_boundary(cpd27607_e, type='exchange', reaction_id='EX_cpd27607_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd27607_e.name = 'N-Acetyl-D-Galactosamine exchange'
iCdR700.reactions.EX_cpd27607_e.annotation['sbo'] = 'SBO:0000627'

cpd15390_c = cobra.Metabolite(
    'cpd15390_c',
    formula='C8H14NO9P',
    name='N-Acetyl-D-Galactosamine-1-phosphate',
    compartment='cytosol')
cpd15390_c.annotation['sbo'] = 'SBO:0000247'
cpd15390_c.annotation['seed.compound'] = 'cpd15390'
cpd15390_c.annotation['bigg.metabolite'] = 'acgal1p'
cpd15390_c.annotation['kegg.compound'] = 'C18060'

cpd00002_c = iCdR700.metabolites.cpd00002_c # ATP
cpd00008_c = iCdR700.metabolites.cpd00008_c # ADP
cpd00067_c = iCdR700.metabolites.cpd00067_c # H

rxn46837_c = cobra.Reaction('rxn46837_c')
rxn46837_c.name = 'ATP:N-acetyl-D-galactosamine 1-phosphotransferase'
rxn46837_c.gene_reaction_rule = '645463.3.peg.2802 and 645463.3.peg.3106 and 645463.3.peg.3106'
rxn46837_c.lower_bound = 0.
rxn46837_c.upper_bound = 1000.
rxn46837_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd27607_e: -1.0,
    cpd00008_c: 1.0,
    cpd00067_c: 1.0,
    cpd15390_c: 1.0
})
rxn46837_c.annotation['sbo'] = 'SBO:0000185' # transport
rxn46837_c.annotation['seed.reaction'] = 'rxn46837'
rxn46837_c.annotation['kegg.reaction'] = 'R08967'
rxn46837_c.annotation['bigg.reaction'] = 'ACGALK'
iCdR700.add_reactions([rxn46837_c])

cpd00062_c = iCdR700.metabolites.cpd00062_c # UTP
cpd00012_c = iCdR700.metabolites.cpd00012_c # Phosphate
cpd00175_c = iCdR700.metabolites.cpd00175_c # UDP-N-acetyl-D-galactosamine

rxn17713_c = cobra.Reaction('rxn17713_c')
rxn17713_c.name = 'UTP-monosaccharide-1-phosphate uridylyltransferase'
rxn17713_c.lower_bound = 0.
rxn17713_c.upper_bound = 1000.
rxn17713_c.add_metabolites({
    cpd00062_c: -1.0,
    cpd15390_c: -1.0,
    cpd00012_c: 1.0,
    cpd00175_c: 1.0
})
rxn17713_c.annotation['sbo'] = 'SBO:0000176' # metabolic
rxn17713_c.annotation['seed.reaction'] = 'rxn17713'
rxn17713_c.annotation['kegg.reaction'] = 'R10183'
rxn17713_c.annotation['bigg.reaction'] = 'GALT'
iCdR700.add_reactions([rxn17713_c])

save_model(iCdR700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700', media=mdm)


Ignoring reaction 'rxn02094_c' since it already exists.
Ignoring reaction 'rxn46837_c' since it already exists.
Ignoring reaction 'rxn17713_c' since it already exists.


### Matching Biolog Results

In [45]:
iCdR700 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.json')

In [46]:
import copy

# Read in curated universal reacton bag and annotate
universal_GramPos = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/GramPos.sbml')
universal_GramPos.reactions.rxn13782_c.remove_from_model(remove_orphans=True)
universal_GramPos.reactions.rxn13783_c.remove_from_model(remove_orphans=True)
universal_GramPos.reactions.rxn13784_c.remove_from_model(remove_orphans=True)
#universal_GramPos.reactions.rxn11920_c.name = 'Poly-beta-hydroxybutyrate (PHB) synthease'
universal_GramPos.reactions.rxn12188_c.name = '2-deoxyglucose-6-phosphatase'

for cpd in universal_GramPos.metabolites:
    cpd.annotation['sbo'] = 'SBO:0000247'
    cpd.annotation['seed.compound'] = cpd.id.split('_')[0]    
    cpd.compartment = cpd.compartment.lower()

for rxn in universal_GramPos.reactions:
    rxn.annotation['sbo'] = 'SBO:0000176'
    rxn.annotation['seed.reaction'] = rxn.id.split('_')[0]

'GramPositive.modeltemplate' is not a valid SBML 'SId'.
No objective coefficients in model. Unclear what should be optimized


In [47]:
# R20291-specific

# Glycogen
rxn12627_c = copy.deepcopy(universal_GramPos.reactions.rxn12627_c)
iCdR700.add_reactions([rxn12627_c])
iCdR700.reactions.rxn12627_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn12627_c.annotation['seed.reaction'] = 'rxn12627'
cpd00155_e = iCdR700.metabolites.cpd00155_e
iCdR700.add_boundary(cpd00155_e, type='exchange', reaction_id='EX_cpd00155_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00155_e.annotation['sbo'] = 'SBO:0000627'

# Glutathione
rxn12539_c = copy.deepcopy(universal_GramPos.reactions.rxn12539_c)
iCdR700.add_reactions([rxn12539_c])
iCdR700.reactions.rxn12539_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn12539_c.annotation['seed.reaction'] = 'rxn12539'
iCdR700.metabolites.cpd00042_e.name = 'Glutathione'
cpd00042_e = iCdR700.metabolites.cpd00042_e
iCdR700.add_boundary(cpd00042_e, type='exchange', reaction_id='EX_cpd00042_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00042_e.annotation['sbo'] = 'SBO:0000627'

# beta-Alanine
rxn05495_c = copy.deepcopy(universal_GramPos.reactions.rxn05495_c)
iCdR700.add_reactions([rxn05495_c])
iCdR700.reactions.rxn05495_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05495_c.annotation['seed.reaction'] = 'rxn05495'
iCdR700.reactions.rxn05495_c.annotation['bigg.reaction'] = ['BALAt2pp','BALAt2r','ala-Bt2']
cpd00085_e = iCdR700.metabolites.cpd00085_e
iCdR700.add_boundary(cpd00085_e, type='exchange', reaction_id='EX_cpd00085_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00085_e.annotation['sbo'] = 'SBO:0000627'

# Agmatine
rxn12597_c = copy.deepcopy(universal_GramPos.reactions.rxn12597_c)
iCdR700.add_reactions([rxn12597_c])
iCdR700.reactions.rxn12597_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn12597_c.annotation['seed.reaction'] = 'rxn12597'
cpd00152_e = iCdR700.metabolites.cpd00152_e
iCdR700.add_boundary(cpd00152_e, type='exchange', reaction_id='EX_cpd00152_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00152_e.annotation['sbo'] = 'SBO:0000627'

# Glycerol
rxn12529_c = copy.deepcopy(universal_GramPos.reactions.rxn12529_c)
iCdR700.add_reactions([rxn12529_c])
iCdR700.reactions.rxn12529_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn12529_c.annotation['seed.reaction'] = 'rxn12529'
iCdR700.reactions.rxn12529_c.annotation['bigg.reaction'] = 'GLYCt2'
cpd00100_e = iCdR700.metabolites.cpd00100_e
iCdR700.add_boundary(cpd00100_e, type='exchange', reaction_id='EX_cpd00100_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00100_e.annotation['sbo'] = 'SBO:0000627'

# Choline
rxn09692_c = copy.deepcopy(universal_GramPos.reactions.rxn09692_c)
iCdR700.add_reactions([rxn09692_c])
iCdR700.reactions.rxn09692_c.name = 'Choline symporter'
iCdR700.reactions.rxn09692_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn09692_c.annotation['seed.reaction'] = 'rxn09692'
iCdR700.reactions.rxn09692_c.annotation['bigg.reaction'] = ['CHLt2','CHLt2pp','CHLt2r','HNM1']
cpd00098_e = iCdR700.metabolites.cpd00098_e
iCdR700.add_boundary(cpd00098_e, type='exchange', reaction_id='EX_cpd00098_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00098_e.annotation['sbo'] = 'SBO:0000627'

In [50]:
# Ala-ala
cpd00002_c = iCdR700.metabolites.cpd00002_c # ATP
cpd00001_c = iCdR700.metabolites.cpd00001_c # H2O
cpd00008_c = iCdR700.metabolites.cpd00008_c # ADP
cpd00009_c = iCdR700.metabolites.cpd00009_c # Phosphate
cpd00067_c = iCdR700.metabolites.cpd00067_c # H+
cpd00731_c = iCdR700.metabolites.cpd00731_c # Ala-ala
cpd00731_e = cobra.Metabolite(
    'cpd00731_e',
    formula='C6H12N2O3',
    name='Ala-ala',
    compartment='extracellular')
cpd00731_e.annotation['sbo'] = 'SBO:0000247'
cpd00731_e.annotation['seed.compound'] = 'cpd00731'
cpd00731_e.annotation['bigg.metabolite'] = ['alaala', 'diala-L']
cpd00731_e.annotation['kegg.compound'] = 'C00993'

# Transport
rxn18658_c = cobra.Reaction('rxn18658_c')
rxn18658_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
rxn18658_c.name = 'Dipeptide transport via ABC system'
rxn18658_c.bounds = (0., 1000.)
rxn18658_c.add_metabolites({
    cpd00731_e: -1.0, # dipeptide
    cpd00002_c: -1.0, # ATP
    cpd00001_c: -1.0, # H2O
    cpd00731_c: 1.0, # dipeptide
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0 # H+
})
rxn18658_c.annotation['sbo'] = 'SBO:0000176'
rxn18658_c.annotation['seed.reaction'] = 'rxn18658'
rxn18658_c.annotation['bigg.reaction'] = 'ALAALAabcpp'
iCdR700.add_reactions([rxn18658_c])

cpd00731_e = iCdR700.metabolites.cpd00731_e
#iCdR700.add_boundary(cpd00731_e, type='exchange', reaction_id='EX_cpd00731_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00731_e.annotation['sbo'] = 'SBO:0000627'

# Hydrolysis
iCdR700.reactions.rxn07466_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'

#--------------------------------------------------------------------------------------------#

# Phosphorylated hexose sugars
rxn05554_c = copy.deepcopy(universal_GramPos.reactions.rxn05554_c) # Fructose-6-phosphate
rxn05562_c = copy.deepcopy(universal_GramPos.reactions.rxn05562_c) # Glucose-1-phosphate
rxn05563_c = copy.deepcopy(universal_GramPos.reactions.rxn05563_c) # Glucose-6-phosphate
iCdR700.add_reactions([rxn05554_c,rxn05562_c,rxn05563_c])

iCdR700.reactions.rxn05554_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05554_c.annotation['seed.reaction'] = 'rxn05554'
iCdR700.reactions.rxn05554_c.annotation['bigg.reaction'] = 'F6Pt6_i2'

iCdR700.reactions.rxn05562_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05562_c.annotation['seed.reaction'] = 'rxn05562'
iCdR700.reactions.rxn05562_c.annotation['bigg.reaction'] = 'G1Pt6_2'

iCdR700.reactions.rxn05563_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05563_c.annotation['seed.reaction'] = 'rxn05563'
iCdR700.reactions.rxn05563_c.annotation['bigg.reaction'] = 'G6Pt6_2'

cpd00072_e = iCdR700.metabolites.cpd00072_e
#iCdR700.add_boundary(cpd00072_e, type='exchange', reaction_id='EX_cpd00072_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00072_e.annotation['sbo'] = 'SBO:0000627'

cpd00089_e = iCdR700.metabolites.cpd00089_e
iCdR700.add_boundary(cpd00089_e, type='exchange', reaction_id='EX_cpd00089_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00089_e.annotation['sbo'] = 'SBO:0000627'

cpd00079_e = iCdR700.metabolites.cpd00079_e
iCdR700.add_boundary(cpd00079_e, type='exchange', reaction_id='EX_cpd00079_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00079_e.annotation['sbo'] = 'SBO:0000627'

# Galactose
rxn05162_c = copy.deepcopy(universal_GramPos.reactions.rxn05162_c)
iCdR700.add_reactions([rxn05162_c])
iCdR700.reactions.rxn05162_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05162_c.annotation['seed.reaction'] = ''
iCdR700.reactions.rxn05162_c.annotation['bigg.reaction'] = ''
cpd00108_e = iCdR700.metabolites.cpd00108_e
iCdR700.add_boundary(cpd00108_e, type='exchange', reaction_id='EX_cpd00108_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.rxn18658_c.annotation['sbo'] = 'SBO:0000627'

#--------------------------------------------------------------------------------------------#

# Intracellular organic acids

# 4-Hydroxybenzoate
rxn12549_c = copy.deepcopy(universal_GramPos.reactions.rxn12549_c)
iCdR700.add_reactions([rxn12549_c])
iCdR700.reactions.rxn12549_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn12549_c.annotation['seed.reaction'] = 'rxn12549'
iCdR700.reactions.rxn12549_c.annotation['bigg.reaction'] = '4HBZt'
cpd00136_e = iCdR700.metabolites.cpd00136_e
iCdR700.add_boundary(cpd00136_e, type='exchange', reaction_id='EX_cpd00136_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00136_e.annotation['sbo'] = 'SBO:0000627'

# 2-Hydroxybutyrate
rxn12585_c = copy.deepcopy(universal_GramPos.reactions.rxn12585_c) 
iCdR700.add_reactions([rxn12585_c])
iCdR700.reactions.rxn12585_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn12585_c.annotation['seed.reaction'] = 'rxn12585'
iCdR700.reactions.rxn12585_c.annotation['bigg.reaction'] = '2HBt2'
cpd03561_e = iCdR700.metabolites.cpd03561_e
iCdR700.add_boundary(cpd03561_e, type='exchange', reaction_id='EX_cpd03561_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd03561_e.annotation['sbo'] = 'SBO:0000627'

# Fumarate
rxn05561_c = copy.deepcopy(universal_GramPos.reactions.rxn05561_c) 
iCdR700.add_reactions([rxn05561_c])
iCdR700.reactions.rxn05561_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05561_c.annotation['seed.reaction'] = 'rxn05561'
iCdR700.reactions.rxn05561_c.annotation['bigg.reaction'] = ['FUMt2r','FUMt6','U207']
cpd00106_e = iCdR700.metabolites.cpd00106_e
iCdR700.add_boundary(cpd00106_e, type='exchange', reaction_id='EX_cpd00106_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00106_e.annotation['sbo'] = 'SBO:0000627'

# Acetoacetate
rxn10960_c = copy.deepcopy(universal_GramPos.reactions.rxn10960_c) 
iCdR700.add_reactions([rxn10960_c])
iCdR700.reactions.rxn10960_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn10960_c.annotation['seed.reaction'] = 'rxn10960'
iCdR700.reactions.rxn10960_c.annotation['bigg.reaction'] = ['ACACtex','U126']
cpd00142_e = iCdR700.metabolites.cpd00142_e
iCdR700.add_boundary(cpd00142_e, type='exchange', reaction_id='EX_cpd00142_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00142_e.annotation['sbo'] = 'SBO:0000627'

# Shikimate
rxn10827_c = copy.deepcopy(universal_GramPos.reactions.rxn10827_c) 
iCdR700.add_reactions([rxn10827_c])
iCdR700.reactions.rxn10827_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn10827_c.annotation['seed.reaction'] = 'rxn10827'
iCdR700.reactions.rxn10827_c.annotation['bigg.reaction'] = ['SKMt2','SKMt2pp']
iCdR700.reactions.rxn10827_c.name = 'Shikimate transport in via proton symport'
cpd00383_e = iCdR700.metabolites.cpd00383_e
iCdR700.add_boundary(cpd00383_e, type='exchange', reaction_id='EX_cpd00383_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00383_e.annotation['sbo'] = 'SBO:0000627'

# Formate
rxn05559_c = copy.deepcopy(universal_GramPos.reactions.rxn05559_c) 
iCdR700.add_reactions([rxn05559_c])
iCdR700.reactions.rxn05559_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05559_c.annotation['seed.reaction'] = 'rxn05559'
iCdR700.reactions.rxn05559_c.annotation['bigg.reaction'] = ['FORt2','FORt2pp']
cpd00047_e = iCdR700.metabolites.cpd00047_e
iCdR700.add_boundary(cpd00047_e, type='exchange', reaction_id='EX_cpd00047_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00047_e.annotation['sbo'] = 'SBO:0000627'

# Glycolate
rxn05470_c = copy.deepcopy(universal_GramPos.reactions.rxn05470_c) 
iCdR700.add_reactions([rxn05470_c])
iCdR700.reactions.rxn05470_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05470_c.annotation['seed.reaction'] = 'rxn05470'
iCdR700.reactions.rxn05470_c.annotation['bigg.reaction'] = ['GLYCLTt2r','GLYCLTt2rpp']
cpd00139_e = iCdR700.metabolites.cpd00139_e
iCdR700.add_boundary(cpd00139_e, type='exchange', reaction_id='EX_cpd00139_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00139_e.annotation['sbo'] = 'SBO:0000627'

# GABA
rxn05564_c = copy.deepcopy(universal_GramPos.reactions.rxn05564_c) 
iCdR700.add_reactions([rxn05564_c])
iCdR700.reactions.rxn05564_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05564_c.annotation['seed.reaction'] = 'rxn05564'
iCdR700.reactions.rxn05564_c.annotation['bigg.reaction'] = ['ABUTt2','ABUTt2pp','ABUTt2r','GABAt2','PUT4_4','UGA4_1']
cpd00281_e = iCdR700.metabolites.cpd00281_e
iCdR700.add_boundary(cpd00281_e, type='exchange', reaction_id='EX_cpd00281_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00281_e.annotation['sbo'] = 'SBO:0000627'

#--------------------------------------------------------------------------------------------#

# Hydroxylamine
rxn05703_c = copy.deepcopy(universal_GramPos.reactions.rxn05703_c) 
iCdR700.add_reactions([rxn05703_c])
iCdR700.reactions.rxn05703_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05703_c.annotation['seed.reaction'] = 'rxn05703'
cpd00165_e = iCdR700.metabolites.cpd00165_e
iCdR700.add_boundary(cpd00165_e, type='exchange', reaction_id='EX_cpd00165_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00165_e.annotation['sbo'] = 'SBO:0000627'

#--------------------------------------------------------------------------------------------#

# Nucleotide analogs

# Deoxyinosine
rxn09687_c = copy.deepcopy(universal_GramPos.reactions.rxn09687_c) 
iCdR700.add_reactions([rxn09687_c])
iCdR700.reactions.rxn09687_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn09687_c.annotation['seed.reaction'] = 'rxn09687'
iCdR700.reactions.rxn09687_c.annotation['bigg.reaction'] = ['DINSt2','DINSt2pp','U180','U196']
cpd03279_e = iCdR700.metabolites.cpd03279_e
iCdR700.add_boundary(cpd03279_e, type='exchange', reaction_id='EX_cpd03279_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd03279_e.annotation['sbo'] = 'SBO:0000627'

# Inosine
rxn05316_c = copy.deepcopy(universal_GramPos.reactions.rxn05316_c) 
iCdR700.add_reactions([rxn05316_c])
iCdR700.reactions.rxn05316_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05316_c.annotation['seed.reaction'] = 'rxn05316'
iCdR700.reactions.rxn05316_c.annotation['bigg.reaction'] = ['INSt2','INSt2pp','INSt2r','INSt2rpp','U177','U194']
cpd00246_e = iCdR700.metabolites.cpd00246_e
iCdR700.add_boundary(cpd00246_e, type='exchange', reaction_id='EX_cpd00246_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00246_e.annotation['sbo'] = 'SBO:0000627'

# Guanosine
rxn05204_c = copy.deepcopy(universal_GramPos.reactions.rxn05204_c) 
iCdR700.add_reactions([rxn05204_c])
iCdR700.reactions.rxn05204_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05204_c.annotation['seed.reaction'] = 'rxn05204'
iCdR700.reactions.rxn05204_c.annotation['bigg.reaction'] = ['GSNt2','GSNt2m','GSNt2pp','GSNt2r','U175','U192']
cpd00311_e = iCdR700.metabolites.cpd00311_e
iCdR700.add_boundary(cpd00311_e, type='exchange', reaction_id='EX_cpd00311_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00311_e.annotation['sbo'] = 'SBO:0000627'

# Cytidine
rxn05198_c = copy.deepcopy(universal_GramPos.reactions.rxn05198_c) 
iCdR700.add_reactions([rxn05198_c])
iCdR700.reactions.rxn05198_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05198_c.annotation['seed.reaction'] = 'rxn05198'
iCdR700.reactions.rxn05198_c.annotation['bigg.reaction'] = ['CYTDt2','CYTDt2pp','CYTDt2r','CYTDt2rpp','U176','U186','U193']
cpd00367_e = iCdR700.metabolites.cpd00367_e
iCdR700.add_boundary(cpd00367_e, type='exchange', reaction_id='EX_cpd00367_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00367_e.annotation['sbo'] = 'SBO:0000627'

# Deoxyguanosine
rxn09688_c = copy.deepcopy(universal_GramPos.reactions.rxn09688_c) 
iCdR700.add_reactions([rxn09688_c])
iCdR700.reactions.rxn09688_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn09688_c.annotation['seed.reaction'] = 'rxn09688'
iCdR700.reactions.rxn09688_c.annotation['bigg.reaction'] = ['DGSNt2','DGSNt2pp','U181','U197']
cpd00277_e = iCdR700.metabolites.cpd00277_e
iCdR700.add_boundary(cpd00277_e, type='exchange', reaction_id='EX_cpd00277_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00277_e.annotation['sbo'] = 'SBO:0000627'

# Adenosine
rxn05318_c = copy.deepcopy(universal_GramPos.reactions.rxn05318_c) 
iCdR700.add_reactions([rxn05318_c])
iCdR700.reactions.rxn05318_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05318_c.annotation['seed.reaction'] = 'rxn05318'
iCdR700.reactions.rxn05318_c.annotation['bigg.reaction'] = ['ADNt2','ADNt2pp','ADNt2r','ADNt2rpp','U174','U185','U191']
cpd00182_e = iCdR700.metabolites.cpd00182_e
iCdR700.add_boundary(cpd00182_e, type='exchange', reaction_id='EX_cpd00182_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00182_e.annotation['sbo'] = 'SBO:0000627'

# Deoxycytidine
rxn05205_c = copy.deepcopy(universal_GramPos.reactions.rxn05205_c) 
iCdR700.add_reactions([rxn05205_c])
iCdR700.reactions.rxn05205_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05205_c.annotation['seed.reaction'] = 'rxn05205'
iCdR700.reactions.rxn05205_c.annotation['bigg.reaction'] = ['DCYTt2','DCYTt2pp','U183','U189','U199']
cpd00654_e = iCdR700.metabolites.cpd00654_e
iCdR700.add_boundary(cpd00654_e, type='exchange', reaction_id='EX_cpd00654_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00654_e.annotation['sbo'] = 'SBO:0000627'

# Deoxyuridine
rxn09685_c = copy.deepcopy(universal_GramPos.reactions.rxn09685_c) 
iCdR700.add_reactions([rxn09685_c])
iCdR700.reactions.rxn09685_c.name = 'Deoxyuridine symport'
iCdR700.reactions.rxn09685_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn09685_c.annotation['seed.reaction'] = 'rxn09685'
iCdR700.reactions.rxn09685_c.annotation['bigg.reaction'] = ['DURIt2','DURIt2pp','U184','U190','U200']
cpd00412_e = iCdR700.metabolites.cpd00412_e
iCdR700.add_boundary(cpd00412_e, type='exchange', reaction_id='EX_cpd00412_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00412_e.annotation['sbo'] = 'SBO:0000627'

# Deoxyadenosine
rxn05317_c = copy.deepcopy(universal_GramPos.reactions.rxn05317_c) 
iCdR700.add_reactions([rxn05317_c])
iCdR700.reactions.rxn05317_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05317_c.annotation['seed.reaction'] = 'rxn05317'
iCdR700.reactions.rxn05317_c.annotation['bigg.reaction'] = ['DADNt2','DADNt2pp','U182','U188','U198']
cpd00438_e = iCdR700.metabolites.cpd00438_e
iCdR700.add_boundary(cpd00438_e, type='exchange', reaction_id='EX_cpd00438_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00438_e.annotation['sbo'] = 'SBO:0000627'

#--------------------------------------------------------------------------------------------#

# Additional amino acids

# Citrulline
rxn05674_c = copy.deepcopy(universal_GramPos.reactions.rxn05674_c) 
iCdR700.add_reactions([rxn05674_c])
iCdR700.reactions.rxn05674_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05674_c.annotation['seed.reaction'] = 'rxn05674'
iCdR700.reactions.rxn05674_c.annotation['bigg.reaction'] = ['CITR_Lt2','CITR_Lt2ex','CITR_Lt2pp']
cpd00274_e = iCdR700.metabolites.cpd00274_e
iCdR700.add_boundary(cpd00274_e, type='exchange', reaction_id='EX_cpd00274_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00274_e.annotation['sbo'] = 'SBO:0000627'

# D-Glutamate
rxn05672_c = copy.deepcopy(universal_GramPos.reactions.rxn05672_c) 
iCdR700.add_reactions([rxn05672_c])
iCdR700.reactions.rxn05672_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn05672_c.annotation['seed.reaction'] = 'rxn05672'
iCdR700.reactions.rxn05672_c.annotation['bigg.reaction'] = 'GLU-Dt2'
cpd00186_e = iCdR700.metabolites.cpd00186_e
iCdR700.add_boundary(cpd00186_e, type='exchange', reaction_id='EX_cpd00186_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00186_e.annotation['sbo'] = 'SBO:0000627'

# Cystathionine
cpd00424_e = cobra.Metabolite(
    'cpd00424_e',
    formula='C7H14N2O4S',
    name='Cystathionine',
    compartment='extracellular')
cpd00424_e.annotation['sbo'] = 'SBO:0000247'
cpd00424_e.annotation['seed.compound'] = 'cpd00424'
cpd00424_e.annotation['bigg.metabolite'] = 'CST_c'
cpd00424_e.annotation['kegg.compound'] = 'C00542'
iCdR700.add_boundary(cpd00424_e, type='exchange', reaction_id='EX_cpd00424_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00424_e.annotation['sbo'] = 'SBO:0000627'

cpd00424_c = iCdR700.metabolites.cpd00424_c
rxn10398_c = cobra.Reaction('rxn10398_c')
rxn10398_c.name = 'cystathione diffusion'
rxn10398_c.bounds = (-1000., 1000.)
rxn10398_c.add_metabolites({
    cpd00424_e: -1.0,
    cpd00424_c: 1.0 
})
rxn10398_c.annotation['sbo'] = 'SBO:0000176'
rxn10398_c.annotation['seed.reaction'] = 'rxn10398'
rxn10398_c.annotation['bigg.reaction'] = 'CYSTtp'
iCdR700.add_reactions([rxn10398_c])
iCdR700.reactions.rxn10398_c.annotation['sbo'] = 'SBO:0000185'
iCdR700.reactions.rxn10398_c.annotation['seed.reaction'] = 'rxn10398'
iCdR700.reactions.rxn10398_c.annotation['bigg.reaction'] = 'CYSTtp'

# LL-2,6-Diaminopimelate
cpd00504_e = cobra.Metabolite(
    'cpd00504_e',
    formula='C7H14N2O4',
    name='LL-2,6-Diaminopimelate',
    compartment='extracellular')
cpd00504_e.annotation['sbo'] = 'SBO:0000247'
cpd00504_e.annotation['seed.compound'] = 'cpd00504'
cpd00504_e.annotation['bigg.metabolite'] = '26dap_LL'
cpd00504_e.annotation['kegg.compound'] = 'C00666'
iCdR700.add_boundary(cpd00504_e, type='exchange', reaction_id='EX_cpd00504_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00504_e.annotation['sbo'] = 'SBO:0000627'

cpd00001_c = iCdR700.metabolites.cpd00001_c
cpd00002_c = iCdR700.metabolites.cpd00002_c
cpd00008_c = iCdR700.metabolites.cpd00008_c
cpd00009_c = iCdR700.metabolites.cpd00009_c
cpd00067_c = iCdR700.metabolites.cpd00067_c
cpd00504_c = iCdR700.metabolites.cpd00504_c

ID011_c = cobra.Reaction('ID011_c')
ID011_c.name = 'ATP phosphohydrolase (polar-amino-acid-importing)'
ID011_c.gene_reaction_rule = '272563.120.peg.772'
ID011_c.bounds = (0., 1000.)
ID011_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd00002_c: -1.0,
    cpd00504_e: -1.0,
    cpd00504_c: 1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0 
})
iCdR700.add_reactions([ID011_c])
iCdR700.reactions.ID011_c.annotation['sbo'] = 'SBO:0000185'

Ignoring reaction 'rxn18658_c' since it already exists.
Ignoring reaction 'rxn05554_c' since it already exists.
Ignoring reaction 'rxn05562_c' since it already exists.
Ignoring reaction 'rxn05563_c' since it already exists.


ValueError: Boundary reaction 'EX_cpd00089_e' already exists.

In [ ]:
# Guanine
rxn05203_c = copy.deepcopy(universal_GramPos.reactions.rxn05203_c) 
rxn05203_c.name = 'Guanine symport'
rxn05203_c.annotation['sbo'] = 'SBO:0000185'
rxn05203_c.annotation['seed.reaction'] = 'rxn05203'
rxn05203_c.annotation['bigg.reaction'] = ['FCY21_3','FCY22_3','FCY2_3','GUAt2','GUAt2pp','GUAt2r','YGL186C_3']

iCdR700.add_reactions([rxn05203_c])
cpd00207_e = iCdR700.metabolites.cpd00207_e
iCdR700.add_boundary(cpd00207_e, type='exchange', reaction_id='EX_cpd00207_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00207_e.annotation['sbo'] = 'SBO:0000627'

In [63]:
final_geneDict = {'272563.8.peg.2607': '645463.3.peg.2656',
                  '272563.8.peg.2464': '645463.3.peg.2536',
                  '272563.8.peg.2736': '645463.3.peg.2384',
                  '272563.8.peg.1365': '645463.3.peg.1218'}
cobra.manipulation.modify.rename_genes(iCdR700, final_geneDict)

In [64]:
iCdR700_rmGenes = basicCheck(iCdR700)
iCdR700_noGPRblocked = blockedReactions(iCdR700)
iCdR700_free = checkFreeMass(iCdR700)
iCdR700_massImbal = checkBalance(iCdR700)
iCdR700_nogene = missingGPR(iCdR700)

GENRE is underdetermined
GENRE has 3 compartment(s)
GENRE has 700 genes
18.93 minutes doubling time
34.59% reactions are blocked
0.0% metabolites are generated for free
0.0% reactions are mass imbalanced
15.27% reactions without GPRs
24.71% of reactions used in pFBA solution have no GPR


In [40]:
save_model(iCdR700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700', media=mdm)

### Integrate CCDS

In [2]:
iCdR700 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.json')

In [68]:
r20291_genes = {}
with open('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/PATRIC/CdR20291.PATRIC.genes.format.fasta','r') as fasta:
    for line in fasta:
        r20291_genes[line.split('|')[0]] = line.split('|')[1]

In [69]:
success = 0
for gene in iCdR700.genes:
    try:
        gene.annotation['ccds'] = r20291_genes[gene.id]
        success += 1
    except KeyError:
            pass
print('Genes found: ' + str(success))

Genes found: 700


In [70]:
save_model(iCdR700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700', media=mdm)

In [71]:
iCdR700

Name,iCdR700
Memory address,0x07fa78371e9d0
Number of metabolites,1244
Number of reactions,1304
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


### Gene Essentiality

In [75]:
# In GENRE, experimentally essential
dembek_genes = set(['645463.3.peg.1000','645463.3.peg.1001','645463.3.peg.1002','645463.3.peg.1007',
                '645463.3.peg.1016','645463.3.peg.1017','645463.3.peg.102','645463.3.peg.103',
                '645463.3.peg.1032','645463.3.peg.1037','645463.3.peg.1057','645463.3.peg.1070',
                '645463.3.peg.1077','645463.3.peg.1079','645463.3.peg.1084','645463.3.peg.1086',
                '645463.3.peg.1089','645463.3.peg.1096','645463.3.peg.1110','645463.3.peg.1112',
                '645463.3.peg.1125','645463.3.peg.1128','645463.3.peg.1129','645463.3.peg.1130',
                '645463.3.peg.1176','645463.3.peg.118','645463.3.peg.119','645463.3.peg.1196',
                '645463.3.peg.1199','645463.3.peg.1218','645463.3.peg.1223','645463.3.peg.1233',
                '645463.3.peg.1240','645463.3.peg.881','645463.3.peg.898','645463.3.peg.904',
                '645463.3.peg.905','645463.3.peg.906','645463.3.peg.907','645463.3.peg.909',
                '645463.3.peg.910','645463.3.peg.916','645463.3.peg.918','645463.3.peg.919',
                '645463.3.peg.920','645463.3.peg.929','645463.3.peg.930','645463.3.peg.954',
                '645463.3.peg.955','645463.3.peg.956','645463.3.peg.957','645463.3.peg.969',
                '645463.3.peg.971','645463.3.peg.976','645463.3.peg.979','645463.3.peg.980',
                '645463.3.peg.981','645463.3.peg.994','645463.3.peg.998'])

In [213]:
iCdR700.slim_optimize()

190.1607048116364

In [212]:
iCdR700 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.json')

In [97]:
# Computationally essential
from cobra.flux_analysis import single_gene_deletion

mdm = ['EX_cpd00001_e','EX_cpd00065_e','EX_cpd00060_e','EX_cpd00322_e','EX_cpd00129_e','EX_cpd00156_e',
       'EX_cpd00107_e','EX_cpd00084_e','EX_cpd00149_e','EX_cpd00099_e','EX_cpd10515_e','EX_cpd00030_e', 
       'EX_cpd00254_e','EX_cpd00063_e','EX_cpd00205_e','EX_cpd00009_e','EX_cpd00971_e','EX_cpd00242_e',
       'EX_cpd00104_e','EX_cpd00644_e','EX_cpd00263_e','EX_cpd00027_e']
for rxn in iCdR700.boundary:
    if rxn.id in mdm:
        rxn.bounds = (-1000., 1000.)
    else:
        rxn.bounds = (0., 1000.)

deletion_results = single_gene_deletion(iCdR700)

iCdR700_essential = []
for x, row in deletion_results.iterrows():
    if float(row.growth) < 1e-6:
        iCdR700_essential.append((list(x)[0]))
iCdR700_essential = set(iCdR700_essential)
print('iCdR700 essential genes: ' + str(len(iCdR700_essential)))

iCdR700 essential genes: 102


In [99]:
iCdR700_genes = set([x.id for x in iCdR700.genes]).intersection(dembek_genes)
agree_essential = dembek_genes.intersection(iCdR700_essential)
disagree_essential = dembek_genes.difference(iCdR700_essential)
ratio = round((float(len(agree_essential)) / float(len(iCdR700_genes))) * 100., 2)
print('Essential genes in agreement: ' + str(ratio) + '%')

Essential genes in agreement: 16.95%


In [92]:
print('Total: ' + str(len(disagree_essential)))
for gene in disagree_essential: print(gene + '\t' + iCdR700.genes.get_by_id(gene).name)

Total: 51
645463.3.peg.1128	phosphopentomutase (EC 5.4.2.7)
645463.3.peg.1037	lactoylglutathione lyase (EC 4.4.1.5)
645463.3.peg.920	NADP-dependent malic enzyme (EC 1.1.1.40)
645463.3.peg.906	3-isopropylmalate dehydratase small subunit (EC 4.2.1.33)
645463.3.peg.1016	Pyruvate formate-lyase (EC 2.3.1.54)
645463.3.peg.910	D-3-phosphoglycerate dehydrogenase (EC 1.1.1.95)
645463.3.peg.103	Methionine aminopeptidase (EC 3.4.11.18)
645463.3.peg.1007	PTS system, galactitol-specific IIB component (EC 2.7.1.69)
645463.3.peg.1070	Homoserine dehydrogenase (EC 1.1.1.3)
645463.3.peg.881	Agmatinase (EC 3.5.3.11)
645463.3.peg.1196	Cysteine desulfurase (EC 2.8.1.7)
645463.3.peg.969	N-acetylmuramoyl-L-alanine amidase (EC 3.5.1.28)
645463.3.peg.1079	Acetate kinase (EC 2.7.2.1)
645463.3.peg.916	Alkanesulfonates ABC transporter ATP-binding protein
645463.3.peg.998	PTS system, mannose-specific IIA component (EC 2.7.1.191)
645463.3.peg.102	Adenylate kinase (EC 2.7.4.3)
645463.3.peg.1057	Methylglyoxal synthas

In [133]:

import copy

# Read in curated universal reacton bag and annotate
universal_GramPos = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/GramPos.sbml')
universal_GramPos.reactions.rxn13782_c.remove_from_model(remove_orphans=True)
universal_GramPos.reactions.rxn13783_c.remove_from_model(remove_orphans=True)
universal_GramPos.reactions.rxn13784_c.remove_from_model(remove_orphans=True)
#universal_GramPos.reactions.rxn11920_c.name = 'Poly-beta-hydroxybutyrate (PHB) synthease'
universal_GramPos.reactions.rxn12188_c.name = '2-deoxyglucose-6-phosphatase'

for cpd in universal_GramPos.metabolites:
    cpd.annotation['sbo'] = 'SBO:0000247'
    cpd.annotation['seed.compound'] = cpd.id.split('_')[0]    
    cpd.compartment = cpd.compartment.lower()

for rxn in universal_GramPos.reactions:
    rxn.annotation['sbo'] = 'SBO:0000176'
    rxn.annotation['seed.reaction'] = rxn.id.split('_')[0]


'GramPositive.modeltemplate' is not a valid SBML 'SId'.
No objective coefficients in model. Unclear what should be optimized


In [201]:
# Fix bounds
#iCdR700.reactions.rxn00926_c.bounds = (0.,1000.) # tough...
iCdR700.reactions.rxn01021_c.bounds = (0.,1000.)
iCdR700.reactions.rxn00143_c.bounds = (0.,1000.)

# Adenine synthesis
rxn00131_c = copy.deepcopy(universal_GramPos.reactions.rxn00131_c) 
iCdR700.add_reactions([rxn00131_c])
iCdR700.reactions.rxn00131_c.bounds = (0., 1000.)
iCdR700.reactions.rxn00131_c.annotation['sbo'] = 'SBO:0000176'
iCdR700.reactions.rxn00131_c.annotation['seed.reaction'] = 'rxn00131'
iCdR700.reactions.rxn00131_c.annotation['bigg.reaction'] = 'AMP-NUCLEOSID-RXN'
iCdR700.reactions.rxn00131_c.annotation['kegg.reaction'] = 'R00182'


In [202]:
# Fix gene names
iCdR700.genes.get_by_id('645463.3.peg.998').name = 'PTS system, mannose-specific IIA component (EC 2.7.1.191)'
iCdR700.genes.get_by_id('645463.3.peg.1000').name = 'PTS system, mannose-specific IIB component (EC 2.7.1.191)'
iCdR700.genes.get_by_id('645463.3.peg.1001').name = 'PTS system, mannose-specific IIC component'
iCdR700.genes.get_by_id('645463.3.peg.1002').name = 'PTS system, mannose-specific IID component'
iCdR700.genes.get_by_id('645463.3.peg.3332').name = 'PTS system, mannose-specific IIA component (EC 2.7.1.191)'
iCdR700.genes.get_by_id('645463.3.peg.3330').name = 'PTS system, mannose-specific IID component'
iCdR700.genes.get_by_id('645463.3.peg.3333').name = 'PTS system, mannose-specific IIB component (EC 2.7.1.191)'
iCdR700.genes.get_by_id('645463.3.peg.467').name = 'PTS system, mannose-specific IID component'
iCdR700.genes.get_by_id('645463.3.peg.303').name = 'PTS system, mannose-specific IIA component (EC 2.7.1.191)'
iCdR700.genes.get_by_id('645463.3.peg.304').name = 'PTS system, mannose-specific IIB component (EC 2.7.1.191)'
iCdR700.genes.get_by_id('645463.3.peg.466').name = 'PTS system, mannose-specific IIC component'
iCdR700.genes.get_by_id('645463.3.peg.465').name = 'PTS system, mannose-specific IIB component (EC 2.7.1.191)'
iCdR700.genes.get_by_id('645463.3.peg.306').name = 'PTS system, mannose-specific IIB component (EC 2.7.1.191)'
iCdR700.genes.get_by_id('645463.3.peg.308').name = 'PTS system, mannose-specific IID component'
iCdR700.genes.get_by_id('645463.3.peg.464').name = 'PTS system, mannose-specific IIA component (EC 2.7.1.191)'
iCdR700.genes.get_by_id('645463.3.peg.307').name = 'PTS system, mannose-specific IIC component'
iCdR700.genes.get_by_id('645463.3.peg.3331').name = 'PTS system, mannose-specific IIC component'
iCdR700.genes.get_by_id('645463.3.peg.2802').name = 'Phosphoenolpyruvate-protein phosphotransferase of PTS system (EC 2.7.3.9) | ptsI'
iCdR700.genes.get_by_id('645463.3.peg.3106').name = 'PTS system, N-acetylglucosamine-specific IIA component (EC 2.7.1.193) / PTS system, N-acetylglucosamine-specific IIB component (EC 2.7.1.193) / PTS system, N-acetylglucosamine-specific IIC component'


In [69]:
mdm = ['EX_cpd00001_e','EX_cpd00065_e','EX_cpd00060_e','EX_cpd00322_e','EX_cpd00129_e','EX_cpd00156_e',
       'EX_cpd00107_e','EX_cpd00084_e','EX_cpd00149_e','EX_cpd00099_e','EX_cpd10515_e','EX_cpd00030_e', 
       'EX_cpd00254_e','EX_cpd00063_e','EX_cpd00205_e','EX_cpd00009_e','EX_cpd00971_e','EX_cpd00242_e',
       'EX_cpd00104_e','EX_cpd00644_e','EX_cpd00263_e','EX_cpd00027_e']
for rxn in iCdR700.boundary:
    if rxn.id in mdm:
        rxn.bounds = (-1000., 1000.)
    else:
        rxn.bounds = (0., 1000.)

In [218]:
save_model(iCdR700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700')

In [239]:
strR20291_only = ['645463.3.peg.56','645463.3.peg.57','645463.3.peg.58','645463.3.peg.59','645463.3.peg.217',
               '645463.3.peg.237','645463.3.peg.239','645463.3.peg.257','645463.3.peg.258','645463.3.peg.259',
               '645463.3.peg.261','645463.3.peg.262','645463.3.peg.326','645463.3.peg.327','645463.3.peg.328',
               '645463.3.peg.348','645463.3.peg.357','645463.3.peg.358','645463.3.peg.359','645463.3.peg.381',
               '645463.3.peg.382','645463.3.peg.400','645463.3.peg.404','645463.3.peg.409','645463.3.peg.410',
               '645463.3.peg.432','645463.3.peg.434','645463.3.peg.436','645463.3.peg.457','645463.3.peg.532',
               '645463.3.peg.535','645463.3.peg.536','645463.3.peg.538','645463.3.peg.541','645463.3.peg.542',
               '645463.3.peg.543','645463.3.peg.545','645463.3.peg.550','645463.3.peg.551','645463.3.peg.552',
               '645463.3.peg.555','645463.3.peg.556','645463.3.peg.561','645463.3.peg.582','645463.3.peg.592',
               '645463.3.peg.604','645463.3.peg.621','645463.3.peg.632','645463.3.peg.650','645463.3.peg.659',
               '645463.3.peg.663','645463.3.peg.724','645463.3.peg.742','645463.3.peg.768','645463.3.peg.927',
               '645463.3.peg.936','645463.3.peg.967','645463.3.peg.1027','645463.3.peg.1030',
               '645463.3.peg.1139','645463.3.peg.1140','645463.3.peg.1141','645463.3.peg.1145',
               '645463.3.peg.1150','645463.3.peg.1157','645463.3.peg.1206','645463.3.peg.1342',
               '645463.3.peg.1343','645463.3.peg.1355','645463.3.peg.1402','645463.3.peg.1433',
               '645463.3.peg.1496','645463.3.peg.1497','645463.3.peg.1501','645463.3.peg.1503',
               '645463.3.peg.1504','645463.3.peg.1506','645463.3.peg.1507','645463.3.peg.1508',
               '645463.3.peg.1509','645463.3.peg.1510','645463.3.peg.1511','645463.3.peg.1512',
               '645463.3.peg.1514','645463.3.peg.1515','645463.3.peg.1517','645463.3.peg.1518',
               '645463.3.peg.1520','645463.3.peg.1521','645463.3.peg.1526','645463.3.peg.1533',
               '645463.3.peg.1538','645463.3.peg.1545','645463.3.peg.1546','645463.3.peg.1558',
               '645463.3.peg.1559','645463.3.peg.1561','645463.3.peg.1563','645463.3.peg.1564',
               '645463.3.peg.1565','645463.3.peg.1570','645463.3.peg.1664','645463.3.peg.1676',
               '645463.3.peg.1806','645463.3.peg.1859','645463.3.peg.1867','645463.3.peg.1868',
               '645463.3.peg.1869','645463.3.peg.1870','645463.3.peg.1871','645463.3.peg.1872',
               '645463.3.peg.1873','645463.3.peg.1874','645463.3.peg.1876','645463.3.peg.1877',
               '645463.3.peg.1878','645463.3.peg.1879','645463.3.peg.1880','645463.3.peg.1881',
               '645463.3.peg.1882','645463.3.peg.1883','645463.3.peg.1884','645463.3.peg.1885',
               '645463.3.peg.1887','645463.3.peg.1892','645463.3.peg.1895','645463.3.peg.1898',
               '645463.3.peg.1900','645463.3.peg.1901','645463.3.peg.1902','645463.3.peg.1904',
               '645463.3.peg.1905','645463.3.peg.1906','645463.3.peg.1907','645463.3.peg.1908',
               '645463.3.peg.2037','645463.3.peg.2056','645463.3.peg.2059','645463.3.peg.2085',
               '645463.3.peg.2086','645463.3.peg.2093','645463.3.peg.2241','645463.3.peg.2344',
               '645463.3.peg.2345','645463.3.peg.2426','645463.3.peg.2429','645463.3.peg.2430',
               '645463.3.peg.2435','645463.3.peg.2453','645463.3.peg.2454','645463.3.peg.2651',
               '645463.3.peg.2652','645463.3.peg.2677','645463.3.peg.2716','645463.3.peg.2866',
               '645463.3.peg.2868','645463.3.peg.2869','645463.3.peg.2874','645463.3.peg.2877',
               '645463.3.peg.2894','645463.3.peg.2923','645463.3.peg.2934','645463.3.peg.2966',
               '645463.3.peg.2997','645463.3.peg.3073','645463.3.peg.3074','645463.3.peg.3075',
               '645463.3.peg.3076','645463.3.peg.3077','645463.3.peg.3078','645463.3.peg.3116',
               '645463.3.peg.3129','645463.3.peg.3131','645463.3.peg.3132','645463.3.peg.3133',
               '645463.3.peg.3135','645463.3.peg.3157','645463.3.peg.3160','645463.3.peg.3165',
               '645463.3.peg.3166','645463.3.peg.3167','645463.3.peg.3168','645463.3.peg.3169',
               '645463.3.peg.3170','645463.3.peg.3171','645463.3.peg.3172','645463.3.peg.3173',
               '645463.3.peg.3174','645463.3.peg.3176','645463.3.peg.3177','645463.3.peg.3178',
               '645463.3.peg.3180','645463.3.peg.3181','645463.3.peg.3182','645463.3.peg.3183',
               '645463.3.peg.3184','645463.3.peg.3185','645463.3.peg.3197','645463.3.peg.3213',
               '645463.3.peg.3214','645463.3.peg.3248','645463.3.peg.3356','645463.3.peg.3357',
               '645463.3.peg.3378','645463.3.peg.3380','645463.3.peg.3382','645463.3.peg.3475',
               '645463.3.peg.3479','645463.3.peg.3480','645463.3.peg.3481','645463.3.peg.3482',
               '645463.3.peg.3483','645463.3.peg.3484','645463.3.peg.3485','645463.3.peg.3486',
               '645463.3.peg.3487','645463.3.peg.3488','645463.3.peg.3489','645463.3.peg.3490',
               '645463.3.peg.3492','645463.3.peg.3493','645463.3.peg.3568','645463.3.peg.3665',
               '645463.3.peg.3669','645463.3.peg.3677','645463.3.peg.3683','645463.3.peg.3686',
               '645463.3.peg.3691']

draft_PATRIC = cobra.io.load_json_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/PATRIC/CdR20291.draft.json')
for cpd in draft_PATRIC.metabolites:
    if cpd.compartment == 'c':
        cpd.compartment = 'cytosol'
    elif cpd.compartment == 'e':
        cpd.compartment = 'extracellular'

test_rxns = []
for gene in strR20291_only:
    try:
        test_rxns += [x.id for x in draft_PATRIC.genes.get_by_id(gene).reactions]
    except:
        pass

for x in test_rxns:
    try:
        rxn = iCdR700.reactions.get_by_id(x)
    except:
        print(x)

yep!
yep!
yep!
yep!
yep!
yep!
yep!
yep!
yep!
yep!
yep!
yep!
yep!
yep!
yep!
yep!
yep!
yep!
yep!


In [65]:
iCdR700 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.json')

In [66]:
iCdR700.metabolites.cpd12255_c_c.id = 'cpd12255_c'
iCdR700.reactions.rxn00139_c.bounds = (-1000., 1000.)

In [ ]:
# Missing exchange reactions
iCdR700.metabolites.cpd11574_e.name = 'Molybdate'
cpd11574_e = iCdR700.metabolites.cpd11574_e
iCdR700.add_boundary(cpd11574_e, type='exchange', reaction_id='EX_cpd11574_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd11574_e.name = 'Molybdate exchange'
iCdR700.reactions.EX_cpd11574_e.annotation['sbo'] = 'SBO:0000627'

iCdR700.metabolites.cpd04097_e.name = 'Lead'
cpd04097_e = iCdR700.metabolites.cpd04097_e
iCdR700.add_boundary(cpd04097_e, type='exchange', reaction_id='EX_cpd04097_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd04097_e.name = 'Lead exchange'
iCdR700.reactions.EX_cpd04097_e.annotation['sbo'] = 'SBO:0000627'

iCdR700.metabolites.cpd01012_e.name = 'Cadmium 2+'
cpd01012_e = iCdR700.metabolites.cpd01012_e
iCdR700.add_boundary(cpd01012_e, type='exchange', reaction_id='EX_cpd01012_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd01012_e.name = 'Cadmium exchange'
iCdR700.reactions.EX_cpd01012_e.annotation['sbo'] = 'SBO:0000627'

iCdR700.metabolites.cpd00531_e.name = 'Mercury 2+'
cpd00531_e = iCdR700.metabolites.cpd00531_e
iCdR700.add_boundary(cpd00531_e, type='exchange', reaction_id='EX_cpd00531_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00531_e.name = 'Mercury exchange'
iCdR700.reactions.EX_cpd00531_e.annotation['sbo'] = 'SBO:0000627'

iCdR700.metabolites.cpd00058_e.name = 'Copper 2+'
cpd00058_e = iCdR700.metabolites.cpd00058_e
iCdR700.add_boundary(cpd00058_e, type='exchange', reaction_id='EX_cpd00058_e', lb=-1000.0, ub=1000.0)
iCdR700.reactions.EX_cpd00058_e.name = 'Copper exchange'
iCdR700.reactions.EX_cpd00058_e.annotation['sbo'] = 'SBO:0000627'

In [71]:
cobra.io.write_sbml_model(iCdR700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.sbml')
cobra.io.save_json_model(iCdR700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.json')

### Testing for Possible Energy Generation Cycles

In [ ]:
# Dissipation reactions

cpd00009_c = iCdR700.metabolites.cpd00009_c # P
cpd00002_c = iCdR700.metabolites.cpd00002_c # ATP 
cpd00008_c = iCdR700.metabolites.cpd00008_c # ADP 
cpd00052_c = iCdR700.metabolites.cpd00052_c # CTP 
cpd00096_c = iCdR700.metabolites.cpd00096_c # CDP 
cpd00038_c = iCdR700.metabolites.cpd00038_c # GTP 
cpd00031_c = iCdR700.metabolites.cpd00031_c # GDP 
cpd00062_c = iCdR700.metabolites.cpd00062_c # UTP 
cpd00014_c = iCdR700.metabolites.cpd00014_c # UDP 
cpd00068_c = iCdR700.metabolites.cpd00068_c # ITP 
cpd00090_c = iCdR700.metabolites.cpd00090_c # IDP 
cpd00067_c = iCdR700.metabolites.cpd00067_c # H 
cpd00004_c = iCdR700.metabolites.cpd00004_c # NADH 
cpd00003_c = iCdR700.metabolites.cpd00003_c # NAD 
cpd00005_c = iCdR700.metabolites.cpd00005_c # NADPH
cpd00006_c = iCdR700.metabolites.cpd00006_c # NADP 
cpd00982_c = iCdR700.metabolites.cpd00982_c # FADH2
cpd00015_c = iCdR700.metabolites.cpd00015_c # FAD 

test_1 = cobra.Reaction('test_1')
test_1.bounds = (0., 1000.)
test_1.add_metabolites({ cpd00002_c: -1.0, cpd00008_c: 1.0, cpd00009_c: 1.0})

test_2 = cobra.Reaction('test_2')
test_2.bounds = (0., 1000.)
test_2.add_metabolites({cpd00052_c: -1.0, cpd00096_c: 1.0, cpd00009_c: 1.0})

test_3 = cobra.Reaction('test_3')
test_3.bounds = (0., 1000.)
test_3.add_metabolites({cpd00038_c: -1.0, cpd00031_c: 1.0, cpd00009_c: 1.0})

test_4 = cobra.Reaction('test_4')
test_4.bounds = (0., 1000.)
test_4.add_metabolites({cpd00062_c: -1.0, cpd00014_c: 1.0, cpd00009_c: 1.0})

test_5 = cobra.Reaction('test_5')
test_5.bounds = (0., 1000.)
test_5.add_metabolites({cpd00068_c: -1.0, cpd00090_c: 1.0, cpd00009_c: 1.0})

test_6 = cobra.Reaction('test_6')
test_6.bounds = (0., 1000.)
test_6.add_metabolites({cpd00004_c: -1.0, cpd00003_c: 1.0, cpd00067_c: 1.0})

test_7 = cobra.Reaction('test_7')
test_7.bounds = (0., 1000.)
test_7.add_metabolites({cpd00005_c: -1.0, cpd00006_c: 1.0, cpd00067_c: 1.0})

test_8 = cobra.Reaction('test_8')
test_8.bounds = (0., 1000.)
test_8.add_metabolites({cpd00982_c: -1.0, cpd00015_c: 1.0, cpd00067_c: 2.0})


In [ ]:
from copy import deepcopy

model = deepcopy(iCdR700)
model.add_reactions([test_1,test_2,test_3,test_4,test_5,test_6,test_7,test_8])
for x in model.boundary: x.lower_bound = 0.
test_rxns = ['test_1','test_2','test_3','test_4','test_5','test_6','test_7','test_8']
edc = set()
for test_rxn in test_rxns:
    
    model.objective = test_rxn
    solution = model.optimize()
    active_rxns = set([rxn.id for rxn in model.reactions if abs(solution.fluxes[rxn.id]) > 1e-6])
    if len(active_rxns) > 0: edc |= active_rxns

edc = edc.difference(set(test_rxns))
edc = edc.difference(set([x.id for x in model.boundary]))
print('Total reactions in possible EDCs:', len(edc))

# Check reversibility 
reversible = []
irreversible = []
for rxn in edc:
    if iCdR700.reactions.get_by_id(rxn).lower_bound == 0.:
        irreversible.append(rxn)
    elif iCdR700.reactions.get_by_id(rxn).upper_bound == 0.:
        irreversible.append(rxn)
    else:
        reversible.append(rxn)
    

In [ ]:
print('Total possible reversibility errors:', len(reversible))
for x in reversible: print(x, iCdR700.reactions.get_by_id(x).name)

In [ ]:
iCdR700.reactions.rxn00799_c.lower_bound = 0.
iCdR700.reactions.rxn05655_c.lower_bound = 0.
iCdR700.reactions.rxn01870_c.lower_bound = 0.
iCdR700.reactions.rxn00293_c.lower_bound = 0.
iCdR700.reactions.rxn00786_c.lower_bound = 0.
iCdR700.reactions.rxn00541_c.lower_bound = 0.
iCdR700.reactions.rxn13150_c.lower_bound = 0.
iCdR700.reactions.rxn02005_c.lower_bound = 0.
iCdR700.reactions.rxn10036_c.lower_bound = 0.
iCdR700.reactions.rxn00606_c.lower_bound = 0.
iCdR700.reactions.rxn00542_c.lower_bound = 0.
iCdR700.reactions.rxn00519_c.lower_bound = 0.
iCdR700.reactions.rxn10183_c.lower_bound = 0.
iCdR700.reactions.rxn01211_c.lower_bound = 0.
iCdR700.reactions.rxn00182_c.lower_bound = 0.

iCdR700.reactions.rxn05209_c.name = 'Sodium hydrogen symporter'

In [ ]:
# Remove incorrect aerobic reactions 
iCdR700.remove_reactions(['rxn02304_c', 'rxn09180_c', 'rxn30075_c'])

# Add other detoxification reaction
cpd00001_c = iCdR700.metabolites.cpd00001_c # H2O
cpd00023_c = iCdR700.metabolites.cpd00023_c # L-Glutamate
cpd00003_c = iCdR700.metabolites.cpd00003_c # NAD
cpd00004_c = iCdR700.metabolites.cpd00004_c # NADH
cpd00024_c = iCdR700.metabolites.cpd00024_c # 2-Oxoglutarate
cpd00067_c = iCdR700.metabolites.cpd00067_c # H+
cpd00013_c = iCdR700.metabolites.cpd00013_c # Ammonia

rxn30075_c = cobra.Reaction('rxn30075_c')
rxn30075_c.name = 'Glutamate dehydrogenase (NAD)'
rxn30075_c.gene_reaction_rule = '645463.3.peg.194'
rxn30075_c.lower_bound = 0.
rxn30075_c.upper_bound = 1000.
rxn30075_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd00003_c: -1.0,
    cpd00023_c: -1.0,
    cpd00024_c: 1.0,
    cpd00004_c: 1.0,
    cpd00067_c: 1.0,
    cpd00013_c: 1.0
})
iCdR700.add_reactions([rxn30075_c])

iCdR700.reactions.rxn30075_c.annotation['sbo'] = 'SBO:0000176' 
iCdR700.reactions.rxn30075_c.annotation['seed.reaction'] = 'rxn30075'
iCdR700.reactions.rxn30075_c.annotation['bigg.reaction'] = 'GLUDx'


In [ ]:
# Incorrect auxotrophies

# Correct proline auxotrophy
iCdR700.remove_reactions(['rxn00471_c'])


In [ ]:
# RNA transcription
iCdR700.remove_reactions(['rna_rxn'])

cpd00002_c = iCdR700.metabolites.cpd00002_c # ATP
cpd00052_c = iCdR700.metabolites.cpd00052_c # CTP
cpd00062_c = iCdR700.metabolites.cpd00062_c # UTP
cpd00038_c = iCdR700.metabolites.cpd00038_c # GTP
cpd00001_c = iCdR700.metabolites.cpd00001_c # H2O

cpd00008_c = iCdR700.metabolites.cpd00008_c # ADP
cpd00009_c = iCdR700.metabolites.cpd00009_c # Phosphate
cpd00012_c = iCdR700.metabolites.cpd00012_c # PPi

cpd17043_c = iCdR700.metabolites.cpd17043_c # RNA polymer

rna_rxn = cobra.Reaction('rna_rxn')
rna_rxn.name = 'RNA transcription'
rna_rxn.lower_bound = 0.
rna_rxn.upper_bound = 1000.
rna_rxn.add_metabolites({
    cpd00002_c: -1.389,
    cpd00052_c: -0.237,
    cpd00062_c: -0.5,
    cpd00038_c: -0.141,
    cpd00001_c: -1.0,
    cpd17043_c: 1.0,
    cpd00008_c: 2.0,
    cpd00009_c: 2.0,
    cpd00012_c: 1.0
})
iCdR700.add_reactions([rna_rxn])
iCdR700.reactions.rna_rxn.annotation['sbo'] = 'SBO:0000629'  

# DNA replication
iCdR700.remove_reactions(['dna_rxn'])

cpd00115_c = iCdR700.metabolites.cpd00115_c # dATP
cpd00356_c = iCdR700.metabolites.cpd00356_c # dCTP
cpd00357_c = iCdR700.metabolites.cpd00357_c # TTP 
cpd00241_c = iCdR700.metabolites.cpd00241_c # dGTP
cpd00002_c = iCdR700.metabolites.cpd00002_c # ATP
cpd00001_c = iCdR700.metabolites.cpd00001_c # H2O

cpd00008_c = iCdR700.metabolites.cpd00008_c # ADP
cpd00009_c = iCdR700.metabolites.cpd00009_c # Phosphate
cpd00012_c = iCdR700.metabolites.cpd00012_c # PPi

cpd17042_c = iCdR700.metabolites.cpd17042_c # DNA polymer

dna_rxn = cobra.Reaction('dna_rxn')
dna_rxn.name = 'DNA replication'
dna_rxn.lower_bound = 0.
dna_rxn.upper_bound = 1000.
dna_rxn.add_metabolites({
    cpd00115_c: -0.5,
    cpd00356_c: -0.176,
    cpd00357_c: -0.454,
    cpd00241_c: -0.308,
    cpd00002_c: -2.0,
    cpd00001_c: -1.0,
    cpd17042_c: 1.0,
    cpd00008_c: 2.0,
    cpd00009_c: 2.0,
    cpd00012_c: 1.0
})
iCdR700.add_reactions([dna_rxn])
iCdR700.reactions.dna_rxn.annotation['sbo'] = 'SBO:0000629'  

In [1]:
iCdR700

NameError: name 'iCdR700' is not defined

In [ ]:
cobra.io.write_sbml_model(iCdR700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.sbml')
cobra.io.save_json_model(iCdR700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.json')

In [ ]:
from cobra.manipulation.delete import prune_unused_metabolites

iCdG693, unused = prune_unused_metabolites(iCdG693)
iCdR700, unused = prune_unused_metabolites(iCdR700)

iCdG693.reactions.rxn05527_c.name = 'Cytosine symporter'
iCdG693.reactions.rxn05204_c.name = 'Guanosine symporter'
iCdG693.reactions.rxn05318_c.name = 'Adenosine symporter'
iCdG693.reactions.rxn09687_c.name = 'Deoxyinosine symporter'
iCdG693.reactions.rxn05871_c.name = 'Primary_alcohol:NAD+ oxidoreductase'

iCdR700.reactions.rxn05527_c.name = 'Cytosine symporter'
iCdR700.reactions.rxn05204_c.name = 'Guanosine symporter'
iCdR700.reactions.rxn05318_c.name = 'Adenosine symporter'
iCdR700.reactions.rxn09687_c.name = 'Deoxyinosine symporter'
iCdR700.reactions.rxn05871_c.name = 'Primary_alcohol:NAD+ oxidoreductase'


In [ ]:
iCdG693 = cobra.io.load_json_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG693.json')
iCdR700 = cobra.io.load_json_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.json')

iCdG693.metabolites.cpd00419_c.name = 'Pyridoxamine'
iCdR700.metabolites.cpd00419_c.name = 'Pyridoxamine'

iCdG693.genes.get_by_id('272563.8.peg.262').name = 'Putative CDP-glycerol:Poly(Glycerophosphate) glycerophosphotransferase'

# Fix residual R20291 gene
import copy

rxn30075_c = copy.deepcopy(iCdG693.reactions.rxn30075_c)
cobra.manipulation.delete.remove_genes(iCdG693, [iCdG693.genes.get_by_id('645463.3.peg.194')])

gene = cobra.Gene('272563.120.peg.195')
rxn30075_c.gene_reaction_rule = '272563.120.peg.195'
iCdG693.add_reactions([rxn30075_c])


iCdG693.genes.get_by_id('272563.120.peg.195').name = 'NAD-specific glutamate dehydrogenase'
iCdG693.genes.get_by_id('272563.120.peg.195').annotation['sbo'] = 'SBO:0000243'
iCdG693.genes.get_by_id('272563.120.peg.195').annotation['refseq'] = 'AJP09876.1'
iCdG693.genes.get_by_id('272563.120.peg.195').annotation['refseq.locustag'] = 'CDIF630_00301'
iCdG693.genes.get_by_id('272563.120.peg.195').annotation['ccds'] = 'ATGTCAGGAAAAGATGTAAATGTCTTCGAGATGGCGCAATCTCAAGTAAAAAATGCATGTGATAAATTAGGTATGGAACCAGCAGTTTATGAATTATTAAAAGAACCTATGAGAGTTATAGAGGTTTCAATTCCAGTAAAAATGGATGATGGTTCTATAAAAACTTTTAAAGGATTTAGATCACAACATAATGATGCAGTAGGGCCAACAAAAGGTGGAATAAGATTCCATCAAAATGTTTCAAGAGACGAAGTAAAAGCTTTATCTATATGGATGACTTTCAAGTGTTCTGTAACAGGTATACCATATGGTGGAGGTAAAGGTGGAATAATAGTAGATCCATCAACTTTATCTCAAGGTGAATTAGAAAGATTAAGTAGAGGATACATAGATGGAATATATAAATTAATAGGTGAGAAAGTTGACGTTCCTGCACCAGACGTAAATACTAATGGTCAAATAATGTCTTGGATGGTTGATGAGTACAATAAATTAACTGGACAAAGTTCTATAGGTGTTATAACTGGTAAGCCAGTTGAATTTGGTGGTTCTTTAGGAAGAACAGCTGCTACTGGATTTGGTGTTGCTGTTACTGCAAGAGAAGCTGCTGCTAAATTAGGAATAGATATGAAAAAAGCTAAAATAGCTGTTCAAGGTATAGGAAACGTTGGTTCTTATACAGTTCTTAACTGTGAAAAACTTGGTGGTACTGTTGTAGCTATGGCTGAATGGTGTAAATCAGAAGGTTCTTATGCTATATACAATGAAAATGGTTTAGATGGTCAAGCTATGTTAGATTATATGAAAGAACATGGTAACTTATTAAACTTCCCAGGAGCTAAGAGAATATCTTTAGAAGAGTTCTGGGCTTCAGATGTTGATATAGTAATACCAGCTGCATTAGAAAACTCTATAACTAAAGAAGTTGCTGAATCTATAAAAGCTAAATTAGTTTGTGAGGCTGCTAATGGACCAACTACTCCAGAGGCTGATGAAGTATTTGCTGAAAGAGGAATAGTTCTTACTCCAGATATATTAACTAATGCTGGTGGAGTTACAGTTTCTTACTTCGAGTGGGTACAAAACTTATATGGATACTACTGGTCAGAAGAAGAAGTAGAGCAAAAAGAAGAAATAGCTATGGTTAAAGCATTTGAATCTATTTGGAAAATCAAAGAAGAGTACAATGTTACAATGAGAGAAGCTGCTTACATGCATTCAATTAAGAAAGTTGCTGAAGCTATGAAATTAAGAGGATGGTACTAA'
iCdG693.genes.get_by_id('272563.120.peg.195').annotation['uniprot'] = 'DHE2_CLODI'
iCdG693.genes.get_by_id('272563.120.peg.195').annotation['ec_number'] = '1.4.1.2'


cobra.io.write_sbml_model(iCdG693, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG693.sbml')
cobra.io.save_json_model(iCdG693, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG693.json')
cobra.io.write_sbml_model(iCdR700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.sbml')
cobra.io.save_json_model(iCdR700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.json')
